# PROJET INSEE  - RÉSUMÉ SYNTHÉTIQUE DE ANALYSE DE LA MOBILITÉ TRANSFRONTALIÈRE DANS LA RÉGION GRAND-EST

# RÉSUMÉ — PARTIE I : Préparation de la Base de Données

---

## 1. Objectif de la partie

Fusionner deux fichiers du Recensement de la Population 2021 (INSEE) — **MOBPRO** (mobilités professionnelles) et **INDCVI** (caractéristiques individuelles) — pour constituer une base de données unique permettant l'analyse économétrique des déterminants du travail transfrontalier dans la région Grand Est.

**Enjeu central** : Les deux fichiers ne partagent *aucun identifiant individuel commun*. L'appariement classique par clé unique (NIR, identifiant fiscal…) est donc impossible, imposant le recours à une méthode d'appariement déterministe sur clé composite.

---

## 2. Sources de données initiales

| Fichier | Contenu | Observations brutes | Colonnes |
|---------|---------|---------------------|----------|
| **MOBPRO** | Lieu de travail et mobilité professionnelle | 553 997 | 32 |
| **INDCVI** | Caractéristiques sociodémographiques détaillées | 1 326 580 | 94 |

Après filtrage sur les **actifs occupés** :

| Fichier | Critère de filtrage | Observations retenues |
|---------|---------------------|----------------------|
| MOBPRO | `STAT ∈ {10, 21, 22, 23}` | 531 298 (95.9%) |
| INDCVI | `STATR ∈ {'1', '2'}` | 531 425 (40.1%) |

---

## 3. Diagnostic préalable : problèmes identifiés

### 3.1. Variables communes

L'exploration identifie **23 variables communes** entre les deux bases, réparties en trois catégories :

| Catégorie | Nb variables | Exemples |
|-----------|--------------|----------|
| **Parfaites** (score = 1.0) | 8 | SEXE, IMMI, INATC, MOCO, NA5, REGION, METRODOM, ARM |
| **OK après correction** | 8 | AGEREVQ, LPRM, NPERR, VOIT, TYPL, TYPMR, STOCD, DEROU |
| **À éviter** (score < 0.5) | 7 | DIPL, EMPL, GS, TP, TRANS, ILT, IPONDI |

**Types de problèmes détectés** :
- **Padding** : codes "08" vs "8" (DIPL, IPONDI)
- **Types différents** : int64 vs object (GS, ILT, TP, TRANS, EMPL)
- **Tokens spéciaux INSEE** : Z, ZZ, ZZZZZZZZZ (valeurs manquantes codées)
- **Overlap faible** : modalités non communes entre les deux bases

### 3.2. Anomalie géographique majeure : le cas des Ardennes (08)

L'analyse géographique révèle une **incohérence critique** :

| Base | Département 08 (Ardennes) | Départements 80-85 |
|------|---------------------------|---------------------|
| **MOBPRO** | ❌ **ABSENT** | 5 586 observations |
| **INDCVI** | ✅ 5 002 observations | ❌ Absents |

**Interprétation** : Les codes 80-85 dans MOBPRO ne correspondent à aucun département du Grand Est. Il s'agit vraisemblablement d'une erreur de codage ou d'extraction dans le fichier source.

**Conséquence** : Le département des Ardennes (08), pourtant **frontalier avec la Belgique**, a dû être **exclu** de l'analyse. Cette exclusion constitue une **limite importante** du projet, car elle réduit la couverture des zones frontalières belges.

**Décision** : Filtrage géographique strict sur **9 départements** (au lieu de 10) :

```
10 (Aube), 51 (Marne), 52 (Haute-Marne), 54 (Meurthe-et-Moselle),
55 (Meuse), 57 (Moselle), 67 (Bas-Rhin), 68 (Haut-Rhin), 88 (Vosges)
```

### 3.3. Problème des IRIS non renseignés

La variable IRIS (Îlots Regroupés pour l'Information Statistique) présente une particularité structurelle :

| Statut IRIS | Population INDCVI | Signification |
|-------------|-------------------|---------------|
| IRIS valide (9 chiffres) | 53.1% | Communes urbaines découpées en IRIS |
| `IRIS = ZZZZZZZZZ` | 46.9% | Communes rurales non découpées |

**Enjeu** : Sans traitement spécifique, près de la **moitié des observations** serait inappariable au niveau communal, excluant précisément les **populations rurales frontalières** — celles les plus susceptibles d'effectuer des trajets transfrontaliers.

---

## 4. Stratégie d'appariement développée

### 4.1. Choix méthodologique : appariement déterministe strict

| Option | Principe | Décision |
|--------|----------|----------|
| **Probabiliste** | Appariement approximatif avec marge d'incertitude | ❌ Rejetée |
| **Déterministe strict** | Appariement 1:1 uniquement si clé unique des deux côtés | ✅ Retenue |

**Justification** : L'analyse économétrique nécessite une identification individuelle sans ambiguïté des travailleurs transfrontaliers, incompatible avec un appariement probabiliste.

**Règle fondamentale** :
```
Appariement validé ⟺ (Clé unique dans MOBPRO) ∩ (Clé unique dans INDCVI)
```

### 4.2. Architecture à deux niveaux

Pour maximiser la couverture tout en préservant la rigueur, une stratégie **hybride à deux niveaux** a été développée :

| Niveau | Population cible | Variable géographique | Clé d'appariement |
|--------|------------------|----------------------|-------------------|
| **Niveau 1** | IRIS valides (53.1%) | `DEPCOM` (code commune) | DEPCOM + 22 variables |
| **Niveau 2** | IRIS = ZZZZZZZZZ (46.9%) | `CANTVILLE` (canton) | CANTVILLE + 22 variables |

**Pouvoir discriminant** :
- Niveau 1 : ~4 600 communes dans le Grand Est → granularité très fine
- Niveau 2 : 176 cantons → granularité moyenne, compensée par les 22 variables additionnelles

### 4.3. Reconstruction des variables géographiques

Les variables géographiques nécessaires ont dû être reconstruites :

| Variable | Base | Source | Règle de reconstruction |
|----------|------|--------|-------------------------|
| `DEPCOM` | INDCVI | IRIS | `DEPCOM = IRIS[:5]` (extraction des 5 premiers caractères) |
| `DEPCOM` | MOBPRO | COMMUNE | Copie directe |
| `CANTVILLE` | INDCVI | Native | Aucune reconstruction nécessaire |
| `CANTVILLE` | MOBPRO | COG 2022 | Jointure `COMMUNE → COM → CAN` via table INSEE |

### 4.4. Variables de la clé composite (22 variables)

```
AGEREVQ, SEXE, DIPL, EMPL, GS, IMMI, INATC, LPRM, MOCO, NPERR, 
TYPMR, VOIT, STOCD, TP, TRANS, TYPL, NA5, REGION, METRODOM, 
ARM, DEROU, IPONDI
```

**Rôle clé d'IPONDI** : Le poids de l'individu (jusqu'à 15 décimales) joue un rôle de quasi-identifiant, permettant de discriminer les individus au sein d'un même territoire géographique.

---

## 5. Résultats de l'appariement

### 5.1. Statistiques globales

| Métrique | Valeur | Objectif |
|----------|--------|----------|
| **Base appariée** | 494 483 observations | — |
| **Taux de couverture** | 93.0% | > 80% ✅ |
| **Pertes** | 36 942 observations (7.0%) | < 20% ✅ |

### 5.2. Détail par niveau

| Niveau | Population cible | Appariements réussis | Taux |
|--------|------------------|----------------------|------|
| **Niveau 1** (DEPCOM) | 281 945 | 255 859 | 90.7% |
| **Niveau 2** (CANTVILLE) | 249 480 | 238 624 | 95.6% |

### 5.3. Sources des pertes

| Cause | MOBPRO | INDCVI |
|-------|--------|--------|
| **Collisions de clés** (clés non uniques) | 73 628 | 36 823 |
| **Clés non matchées** | 238 626 | 127 |

**Profil à risque de collision** : Homme, 50 ans, CAP, salarié, ouvrier, commune peuplée (profils fréquents → probabilité de collision plus élevée).

---

## 6. Analyse des biais de sélection

### 6.1. Méthodologie

Pour vérifier que l'appariement n'introduit pas de biais substantiel, une analyse comparative a été réalisée sur 13 variables entre population initiale et population appariée, utilisant :

| Métrique | Principe | Seuil d'alerte |
|----------|----------|----------------|
| Test du χ² | Significativité statistique | p < 0.05 |
| V de Cramér | Intensité de l'association | ≥ 0.10 |
| Distance de Jensen-Shannon | Divergence entre distributions | ≥ 0.05 |
| Écart maximal (Δ pp) | Différence maximale par modalité | ≥ 5 pp |

### 6.2. Résultats

| Statistique | Résultat |
|-------------|----------|
| Variables testées | 13 |
| Variables significatives (α = 5%) | 10 / 13 |
| Variables **non** significatives | SEXE (p=0.082), DIPL (p=0.058), TP (p=0.788) |

**Intensité des effets** (variables significatives) :

| Variable | V de Cramér | JS Distance | Max |Δ| (pp) |
|----------|-------------|-------------|-----------------|
| Type de logement | 0.029 | 0.025 | **2.19** |
| Type de ménage | 0.027 | 0.023 | 1.27 |
| Nombre de personnes | 0.027 | 0.023 | 1.24 |
| Nombre de voitures | 0.025 | 0.021 | 0.97 |
| Mode de transport | 0.007 | 0.006 | 0.61 |
| Lieu de travail | 0.007 | 0.006 | 0.61 |
| Âge quinquennal | 0.015 | 0.014 | 0.42 |
| Type d'emploi | 0.007 | 0.006 | 0.31 |
| Département | 0.007 | 0.006 | 0.28 |
| Groupe socioprofessionnel | 0.004 | 0.003 | 0.20 |

### 6.3. Interprétation

La significativité statistique observée (10/13 variables) s'explique par la **taille très élevée des échantillons** (> 500 000 observations) et ne reflète pas un biais substantiel :

1. **V de Cramér systématiquement < 0.03** → effets négligeables (seuil conventionnel : 0.10)
2. **Distance JS < 0.025** → distributions quasi-identiques
3. **Écart maximal = 2.2 pp** → concentré sur les variables de structure du ménage
4. **Variables clés préservées** : SEXE, DIPL, TP ne présentent aucune différence significative

**Conclusion** : Aucun biais de sélection majeur. La base appariée est représentative de la population initiale.

---

## 7. Base finale validée

| Caractéristique | Valeur |
|-----------------|--------|
| **Observations** | 494 483 |
| **Variables** | 106 |
| **Départements** | 9 (hors Ardennes) |
| **Fichier** | `BDD_FINAL_CLEAN.csv` |

### Répartition de la variable cible

| Statut | Effectif | Proportion |
|--------|----------|------------|
| **Travailleurs transfrontaliers** | 44 264 | 8.95% |
| **Non-transfrontaliers** | 450 219 | 91.05% |

---

## 8. Forces de la démarche

| Force | Description |
|-------|-------------|
| **Rigueur méthodologique** | Appariement déterministe strict 1:1, sans approximation ni fusion many-to-one |
| **Couverture élevée** | 93% de la population cible, dépassant l'objectif de 80% |
| **Préservation des zones rurales** | Récupération des communes sans IRIS via le Niveau 2, évitant un biais anti-rural de 45% |
| **Traçabilité complète** | Chaque appariement documenté et vérifiable |
| **Absence de biais substantiel** | Validé par analyse statistique multi-critères |
| **Exploitation optimale du COG** | Utilisation des tables officielles INSEE pour la reconstruction géographique |

---

## 9. Limites identifiées

| Limite | Impact | Atténuation |
|--------|--------|-------------|
| **Exclusion des Ardennes (08)** | Perte d'un département frontalier avec la Belgique (~5 000 observations) | Aucune possible sans accès aux données sources corrigées |
| **7% de pertes** | 36 942 observations non appariées | Biais résiduel faible et localisé, validé par analyse |
| **Biais potentiel sur les profils fréquents** | Ménages de grande taille et zones denses légèrement sous-représentés | Écart maximal de 2.2 pp, jugé acceptable |
| **Absence d'identifiant unique** | Impossibilité de valider l'appariement par une source externe | Rigueur méthodologique maximale comme compensation |

---

## 10. Transition vers les étapes suivantes

La base de données finale est validée et prête pour le pipeline économétrique :

| Étape | Objectif | Méthode |
|-------|----------|---------|
| **Partie 2** | Sélection des variables pertinentes | Régression pénalisée Elastic Net |
| **Partie 3** | Estimation des déterminants individuels | Probit binaire simple |
| **Partie 4** | Analyse de l'hétérogénéité genrée | Probit avec interactions genre × covariables |
| **Partie 5** | Modélisation des décisions conjointes | Probit bivarié (coordination intra-couple) |

---

## Annexe : Schéma récapitulatif du pipeline

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                        PIPELINE DE PRÉPARATION                              │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│   MOBPRO brut (554k)              INDCVI brut (1.3M)                        │
│         │                               │                                   │
│         ▼                               ▼                                   │
│   ┌─────────────────────────────────────────────────────────────────┐       │
│   │  ÉTAPE 1 : Filtrage actifs occupés                              │       │
│   │    • MOBPRO : STAT ∈ {10, 21, 22, 23} → 531k                    │       │
│   │    • INDCVI : STATR ∈ {'1', '2'}      → 531k                    │       │
│   └─────────────────────────────────────────────────────────────────┘       │
│         │                               │                                   │
│         ▼                               ▼                                   │
│   ┌─────────────────────────────────────────────────────────────────┐       │
│   │  ÉTAPE 2 : Filtrage géographique (9 départements)               │       │
│   │    ⚠️ Exclusion des Ardennes (08) - anomalie données            │       │
│   └─────────────────────────────────────────────────────────────────┘       │
│         │                               │                                   │
│         ▼                               ▼                                   │
│   ┌─────────────────────────────────────────────────────────────────┐       │
│   │  ÉTAPE 3 : Harmonisation des 22 variables communes              │       │
│   │    • Correction des types                                       │       │
│   │    • Suppression du padding                                     │       │
│   │    • Remplacement des tokens NA (Z, ZZ, ZZZZZZZZZ)              │       │
│   └─────────────────────────────────────────────────────────────────┘       │
│         │                               │                                   │
│         ▼                               ▼                                   │
│   ┌─────────────────────────────────────────────────────────────────┐       │
│   │  ÉTAPE 4 : Reconstruction géographique                          │       │
│   │    • DEPCOM (INDCVI) : extraction depuis IRIS[:5]               │       │
│   │    • CANTVILLE (MOBPRO) : jointure via COG 2022                 │       │
│   └─────────────────────────────────────────────────────────────────┘       │
│         │                               │                                   │
│         ▼                               ▼                                   │
│   ┌─────────────────────────────────────────────────────────────────┐       │
│   │  ÉTAPE 5 : Appariement déterministe strict 1:1                  │       │
│   │    • Niveau 1 : DEPCOM + 22 vars (IRIS valides)     → 90.7%     │       │
│   │    • Niveau 2 : CANTVILLE + 22 vars (IRIS = ZZZ)    → 95.6%     │       │
│   └─────────────────────────────────────────────────────────────────┘       │
│         │                                                                   │
│         ▼                                                                   │
│   ┌─────────────────────────────────────────────────────────────────┐       │
│   │  ÉTAPE 6 : Validation (analyse des biais)                       │       │
│   │    • V de Cramér < 0.03 pour toutes les variables               │       │
│   │    • Écart maximal = 2.2 pp (type de logement)                  │       │
│   │    • SEXE, DIPL, TP : aucune différence significative           │       │
│   └─────────────────────────────────────────────────────────────────┘       │
│         │                                                                   │
│         ▼                                                                   │
│   ┌─────────────────────────────────────────────────────────────────┐       │
│   │  BASE FINALE VALIDÉE                                            │       │
│   │    • 494 483 observations × 106 variables                       │       │
│   │    • Taux de couverture : 93.0%                                 │       │
│   │    • Transfrontaliers : 8.95%                                   │       │
│   └─────────────────────────────────────────────────────────────────┘       │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

---

*Résumé rédigé à partir de l'analyse exhaustive du notebook `1_Préparation_Base_de_donnée.ipynb` (8169 lignes)*

# RÉSUMÉ COMPLET PARTIE 2 : SÉLECTION DES VARIABLES PAR APPRENTISSAGE AUTOMATIQUE

---

### OBJECTIF

Identifier de manière objective et reproductible les variables pertinentes pour expliquer la décision de travailler à l'étranger. Cette étape prépare la spécification économétrique des modèles probit en combinant :
- **Analyse de multicolinéarité** : diagnostics statistiques et arbitrages économiques
- **Régularisation Elastic Net** : sélection automatique par pénalisation des coefficients

**Avertissement méthodologique** : L'Elastic Net est utilisé comme outil de sélection de variables, non comme modèle final d'inférence. L'interprétation des effets marginaux sera réalisée dans les parties suivantes avec les modèles probit.

---

### DONNÉES D'ENTRÉE

| Caractéristique | Valeur |
|-----------------|--------|
| Fichier source | BDD_FINAL_CLEAN.csv |
| Observations | 494 483 |
| Variables initiales | 106 |
| Transfrontaliers (Y=1) | 44 264 (8.95%) |
| Non-transfrontaliers (Y=0) | 450 219 (91.05%) |

**Déséquilibre de classes** : La classe minoritaire représente moins de 10% → stratification obligatoire pour la validation croisée.

---

### ÉTAPE 1 : CRÉATION DES VARIABLES DÉRIVÉES

**Variable cible TRANSFRONTALIER** :
$$Y_i = \mathbb{1}\{\text{ILT} = 7\}$$

Où ILT = 7 indique un lieu de travail hors de France.

**Variables pays de destination** :

| Pays | Code DCFLT | Effectif | % transfrontaliers |
|------|------------|----------|-------------------|
| Luxembourg | LU* | ~31 000 | ~70% |
| Allemagne | AL* | ~8 000 | ~18% |
| Suisse | SU* | ~4 000 | ~9% |
| Belgique | BE* | ~1 000 | ~3% |

---

### ÉTAPE 2 : EXCLUSIONS PRÉALABLES

**25 variables exclues** avant toute modélisation, réparties en 5 catégories :

| Catégorie | Variables | Motif |
|-----------|-----------|-------|
| **Non informatives** (6) | ARM, DEROU, METRODOM, REGION, RECH, TACT | 100% manquantes ou constantes sur le Grand Est |
| **Identifiants** (2) | ID_INDIVIDU, NUMMI | Identifiants techniques |
| **Géographie fine** (6) | COMMUNE, DEPCOM, IRIS, TRIRIS, CANTVILLE, CATIRIS | Cardinalité excessive (>4 600 modalités) |
| **Leakage** (9) | ILT, DCLT, DCFLT, REGLT, WORK_IN_PAYS, WORK_LUXEMBOURG, WORK_ALLEMAGNE, WORK_SUISSE, WORK_BELGIQUE | Définissent directement Y |
| **Endogènes** (1) | TRANS | Mode de transport, déterminé par le lieu de travail |

**Variables conservées** : 87 features (34 numériques + 53 catégorielles)

---

### ÉTAPE 3 : ANALYSE DE MULTICOLINÉARITÉ

#### 3.1 Corrélations numériques (Spearman)

| Indicateur | Seuil | Résultat |
|------------|-------|----------|
| Paires avec \|ρ\| ≥ 0.70 | 0.70 | **17 paires** |
| Corrélation maximale | — | 0.999 (AGED ↔ AGEREV) |

**Paires critiques identifiées** :

| Variable 1 | Variable 2 | ρ | Action |
|------------|------------|---|--------|
| AGED | AGEREV | 0.999 | Panier Âge |
| AGED | ANAI | -0.991 | Panier Âge |
| AGEREV | ANAI | -0.991 | Panier Âge |
| AGEREVQ | ANAI | -0.985 | Panier Âge |
| AGER20 | AGEREVQ | 0.969 | Panier Âge |
| IMMI | NAIDT | -0.915 | Panier Origine |
| COUPLE | STAT_CONJ | 0.822 | Panier Conjugal |
| IMMI | INATC | -0.749 | Panier Origine |

#### 3.2 Associations catégorielles (V de Cramér)

| Indicateur | Seuil | Résultat |
|------------|-------|----------|
| Paires avec V ≥ 0.50 | 0.50 | **665 paires** |
| Associations parfaites (V=1.0) | — | Multiples (équipements logement) |

**Clusters identifiés par CAH** : Les variables catégorielles se regroupent naturellement en blocs thématiques (équipements DOM, structure familiale, emploi).

#### 3.3 Variance Inflation Factor (VIF)

| Seuil | Variables concernées |
|-------|---------------------|
| VIF > 10 (critique) | 11 variables |
| VIF > 5 (élevé) | 6 variables supplémentaires |

---

### ÉTAPE 4 : ARBITRAGES ÉCONOMIQUES PAR PANIERS

**7 paniers de variables redondantes** identifiés, avec arbitrage théorique pour chacun :

#### Panier 1 — Catégorie socioprofessionnelle

| Variable | VIF | Description | Décision |
|----------|-----|-------------|----------|
| **GS** | 6.7 | Groupe socioprofessionnel (6 modalités) | ✅ **Conservée** |
| STAT_GSEC | 5.3 | Statut agrégé | ❌ Exclue (redondante) |

**Justification** : GS est la variable standard en économétrie du travail, offrant une granularité suffisante pour l'interprétation.

#### Panier 2 — Âge

| Variable | VIF | Description | Décision |
|----------|-----|-------------|----------|
| **AGEREV** | 14.2 | Âge révolu en années | ✅ **Conservée** |
| **AGEREV²** | — | Terme quadratique | ✅ **Créée** |
| AGED, AGEREVQ, AGER20, ANAI | >10 | Redondantes | ❌ Exclues |

**Justification** : L'effet de l'âge sur la mobilité transfrontalière est non-linéaire (U inversé). La forme quadratique capture le cycle de vie professionnelle.

#### Panier 3 — Statut professionnel

| Variable | VIF | Description | Décision |
|----------|-----|-------------|----------|
| **EMPL** | 3.2 | Condition d'emploi détaillée | ✅ **Conservée** |
| STAT | 2.8 | Statut agrégé | ❌ Exclue |
| STATR | 2.1 | Statut simplifié | ❌ Exclue |

**Justification** : EMPL distingue CDI, CDD, intérim, apprentissage, indépendant — nuances essentielles pour l'analyse du travail transfrontalier.

#### Panier 4 — Nationalité et immigration

| Variable | VIF | Description | Décision |
|----------|-----|-------------|----------|
| **INATC** | 2.4 | Nationalité (Français/Étranger) | ✅ **Conservée** |
| IMMI | 2.1 | Statut d'immigré | ❌ Exclue |
| NAIDT | 1.8 | Origine DOM-TOM | ❌ Exclue |

**Justification** : Dans le Grand Est, la distinction Français/Étranger est plus pertinente pour la mobilité transfrontalière. NAIDT est quasi-constant (DOM-TOM négligeable dans cette région).

#### Panier 5 — Situation conjugale

| Variable | VIF | Description | Décision |
|----------|-----|-------------|----------|
| **COUPLE** | 7.9 | Vie en couple (binaire) | ✅ **Conservée** |
| STAT_CONJ | 6.0 | Statut conjugal détaillé | ❌ Exclue |

**Justification** : Variable binaire nécessaire pour la cohérence avec le modèle probit bivarié (Partie 5) sur les décisions conjointes des couples.

#### Panier 6 — Structure familiale (47 variables initiales)

| Variable conservée | Justification |
|--------------------|---------------|
| **NENFR** | Nombre d'enfants — effet marginal interprétable |

**Variables exclues** :

| Catégorie | Variables | Motif |
|-----------|-----------|-------|
| Équipements DOM | BAIN, WC, ELEC, EAU, CHOS, CLIM, CUIS, SANIDOM | Quasi-constantes en métropole |
| Taille logement | NBPI, SURF | Proxy du revenu (capturé par GS) |
| Composition ménage | INPER, INPERF, NE17FR, NE24FR | Redondantes avec NENFR |
| Structure familiale | TYPMC, SFM, TYPFC, MOCO_DET, TAF | Redondantes avec NENFR |

#### Panier 7 — Secteur d'activité

| Variable | Modalités | Description | Décision |
|----------|-----------|-------------|----------|
| **NA5** | 5 | Secteur regroupé | ✅ **Conservée** |
| NA17 | 17 | Secteur détaillé | ❌ Exclue (robustesse) |

**Justification** : NA5 capture les grandes tendances sectorielles (industrie vers l'Allemagne, services vers le Luxembourg).

---

### ÉTAPE 5 : DIAGNOSTIC POST-ARBITRAGES

#### Corrélations résiduelles critiques

Après arbitrages, 3 variables présentaient encore des corrélations critiques :

| Paire | ρ | Problème | Action |
|-------|---|----------|--------|
| NENFR_Z ↔ TAF_Z | 1.000 | Redondance parfaite | Supprimer TAF |
| NENFR_0 ↔ TAF_0 | 1.000 | Redondance parfaite | Supprimer TAF |
| HLML_1 ↔ STOCD_22 | 0.982 | Quasi-redondance | Supprimer HLML |
| EMPL ↔ STATR | 0.750 | STATR agrégation d'EMPL | Supprimer STATR |

**Suppressions finales** : TAF, HLML, STATR

#### Matrice finale post-nettoyage

| Métrique | Avant | Après |
|----------|-------|-------|
| Colonnes | 91 | 81 |
| Variables conceptuelles | 20 | 17 |
| Paires avec ρ = 1.0 | 3 | 0 |
| Paires avec ρ > 0.7 | 6 | 2 |

**Recommandation** : Elastic Net (l1_ratio = 0.7) pour gérer les corrélations modérées résiduelles.

---

### ÉTAPE 6 : VARIABLES RETENUES POUR LA MODÉLISATION

#### Issues des arbitrages économiques

| Catégorie | Variable(s) | Forme |
|-----------|-------------|-------|
| Catégorie socioprofessionnelle | GS | Catégorielle (6 modalités) |
| Âge | AGEREV, AGEREV² | Continue + quadratique |
| Statut professionnel | EMPL | Catégorielle |
| Nationalité | INATC | Binaire |
| Situation conjugale | COUPLE | Binaire |
| Enfants | NENFR | Discrète |
| Secteur d'activité | NA5 | Catégorielle (5 modalités) |

#### Hors paniers de redondance (VIF faibles)

| Variable | Description | VIF |
|----------|-------------|-----|
| SEXE | Genre | 1.13 |
| DIPL | Niveau de diplôme | 1.31 |
| DEPT | Département de résidence | 1.02 |
| VOIT | Motorisation du ménage | 1.54 |
| STOCD | Statut d'occupation du logement | 1.65 |
| TP | Temps de travail | 1.14 |
| SANI | Qualité installations sanitaires | 1.21 |
| DNAI | Département de naissance (recodé 3 cat.) | 1.18 |
| ETUD | En cours d'études | 1.09 |

---

### ÉTAPE 7 : ELASTIC NET — OPTIMISATION DES HYPERPARAMÈTRES

#### Configuration

| Paramètre | Valeur |
|-----------|--------|
| Méthode | LogisticRegression (penalty='elasticnet') |
| Solver | saga |
| Validation croisée | 5 folds stratifiés |
| Métrique d'optimisation | AUC-ROC |
| Tolérance | 1e-3 |
| Max iterations | 2 000 |

#### Grille de recherche

| Paramètre | Valeurs testées |
|-----------|-----------------|
| C (régularisation) | [0.01, 0.1, 1.0] |
| l1_ratio | [0.7, 0.9, 1.0] |
| **Combinaisons** | 9 |
| **Total fits** | 45 |

#### Résultats optimaux

| Paramètre | Valeur optimale | Interprétation |
|-----------|-----------------|----------------|
| **C** | 1.0 | Régularisation modérée |
| **l1_ratio** | 0.7 | 70% LASSO + 30% Ridge |
| **AUC-ROC (CV)** | 0.8491 | Excellente discrimination |

**Temps d'exécution** : ~17 minutes (1 038 secondes)

La valeur l1_ratio = 0.7 confirme l'intérêt de la composante Ridge (30%) pour stabiliser la sélection en présence de corrélations résiduelles.

---

### ÉTAPE 8 : RÉSULTATS DE LA SÉLECTION

#### Variables sélectionnées

| Métrique | Valeur |
|----------|--------|
| Features totales | 81 |
| Features sélectionnées (coef ≠ 0) | **81 (100%)** |
| Features éliminées | 0 |

L'Elastic Net a conservé toutes les variables, indiquant que chacune apporte une information marginale non nulle après les arbitrages préalables.

#### Performance sur échantillon test

| Métrique | Seuil 0.50 | Seuil optimal (0.086) | Δ |
|----------|------------|----------------------|---|
| **AUC-ROC** | 0.8448 | 0.8448 | — |
| Accuracy | 91.2% | 72.4% | -18.8 pp |
| Precision | 56.7% | 22.0% | -34.7 pp |
| Recall | 7.2% | 81.6% | **+74.5 pp** |
| F1-Score | 12.7% | 34.6% | +21.9 pp |

**Note** : Le seuil optimal (Youden's J) maximise Sensibilité + Spécificité. Il n'est pertinent que pour un usage prédictif, pas pour l'interprétation économétrique.

#### Matrices de confusion (test, n = 98 897)

**Seuil par défaut (0.50)** :
|  | Prédit 0 | Prédit 1 |
|--|----------|----------|
| Réel 0 | 89 767 | 342 |
| Réel 1 | 8 226 | 562 |

**Seuil optimal (0.086)** :
|  | Prédit 0 | Prédit 1 |
|--|----------|----------|
| Réel 0 | 64 287 | 25 822 |
| Réel 1 | 1 562 | 7 226 |

---

### ÉTAPE 9 : HIÉRARCHIE DES DÉTERMINANTS

#### Top 17 variables conceptuelles par importance

| Rang | Variable | Importance | Modalités | Interprétation |
|:----:|----------|:----------:|:---------:|----------------|
| 1 | **DEPT** | 4.975 | 9 | Localisation géographique = premier déterminant |
| 2 | **AGEREV** | 1.487 | 2 | Cycle de vie professionnelle (effet quadratique) |
| 3 | **NA5** | 1.110 | 5 | Segmentation sectorielle des marchés |
| 4 | **EMPL** | 0.833 | 6 | Conditions d'emploi |
| 5 | **DIPL** | 0.566 | 6 | Accès aux emplois qualifiés à l'étranger |
| 6 | **STOCD** | 0.440 | 5 | Statut d'occupation du logement |
| 7 | **VOIT** | 0.402 | 4 | Motorisation (proxy mobilité) |
| 8 | **SANI** | 0.367 | 4 | Qualité du logement (proxy précarité) |
| 9 | **NENFR** | 0.337 | 6 | Contraintes familiales |
| 10 | **GS** | 0.300 | 6 | Position socioprofessionnelle |
| 11 | **DNAI** | 0.291 | 3 | Origine géographique |
| 12 | **TYPL** | 0.138 | 4 | Type de logement |
| 13 | **INATC** | 0.116 | 2 | Nationalité |
| 14 | **ETUD** | 0.064 | 2 | En cours d'études |
| 15 | **SEXE** | 0.048 | 2 | Genre |
| 16 | **TP** | 0.024 | 2 | Temps de travail |
| 17 | **COUPLE** | 0.011 | 2 | Vie en couple |

---

### ÉTAPE 10 : INTERPRÉTATION ÉCONOMIQUE DES COEFFICIENTS

#### Effets positifs (↑ probabilité transfrontalier)

| Variable | Coefficient | Effet marginal | Interprétation |
|----------|:-----------:|:--------------:|----------------|
| **DEPT_57** (Moselle) | +0.849 | +5.75 pp | Proximité immédiate du Luxembourg |
| **AGEREV** | +0.737 | +4.99 pp | L'expérience est valorisée à l'étranger |
| **DEPT_68** (Haut-Rhin) | +0.539 | +3.65 pp | Proximité Suisse/Allemagne (Bâle, Fribourg) |
| **DEPT_54** (M.-et-Moselle) | +0.410 | +2.78 pp | Accès au bassin luxembourgeois |
| **NA5_GU** (Services) | +0.235 | +1.59 pp | Tertiaire mobile, forte demande au Luxembourg |
| **EMPL_16** (CDI) | +0.178 | +1.21 pp | Stabilité contractuelle facilite les navettes |
| **DNAI_NE_ETRANGER** | +0.169 | +1.15 pp | Réseaux, compétences linguistiques |
| **NA5_BE** (Construction) | +0.163 | +1.11 pp | BTP dynamique au Luxembourg |
| **DIPL_18** (Bac+5) | +0.150 | +1.02 pp | Différentiel salarial attractif pour les qualifiés |
| **STOCD_10** (Propriétaire) | +0.124 | +0.84 pp | Ancrage résidentiel compatible avec navettes |
| **DEPT_67** (Bas-Rhin) | +0.121 | +0.82 pp | Proximité Allemagne (Kehl, Offenburg) |
| **INATC_2** (Étranger) | +0.116 | +0.79 pp | Liens avec le pays d'origine voisin |
| **NENFR_Z** (Sans enfant) | +0.108 | +0.73 pp | Absence de contrainte familiale |

#### Effets négatifs (↓ probabilité transfrontalier)

| Variable | Coefficient | Effet marginal | Interprétation |
|----------|:-----------:|:--------------:|----------------|
| **DEPT_51** (Marne) | -1.122 | -7.60 pp | Éloignement majeur des frontières (Reims) |
| **AGEREV_sq** | -0.751 | -5.09 pp | Effet quadratique : décroissance après 40-45 ans |
| **DEPT_10** (Aube) | -0.722 | -4.89 pp | Département enclavé, sans accès frontalier |
| **DEPT_88** (Vosges) | -0.633 | -4.29 pp | Relief montagneux, accessibilité limitée |
| **DEPT_52** (Haute-Marne) | -0.513 | -3.48 pp | Rural, éloigné des axes transfrontaliers |
| **NA5_OQ** (Administration) | -0.379 | -2.57 pp | Emplois publics ancrés sur le territoire national |
| **SANI_X** (Sans sanitaires) | -0.257 | -1.74 pp | Proxy de précarité, frein à la mobilité |
| **NA5_AZ** (Agriculture) | -0.230 | -1.56 pp | Activité agricole non délocalisable |
| **STOCD_22** (Logé gratuit) | -0.228 | -1.55 pp | Situation résidentielle précaire |
| **EMPL_21** (Indépendant) | -0.204 | -1.38 pp | Activité ancrée localement |
| **VOIT_0** (Sans voiture) | -0.176 | -1.19 pp | Mobilité contrainte |
| **EMPL_22** (Employeur) | -0.126 | -0.86 pp | Chef d'entreprise, activité liée au territoire |
| **INATC_1** (Français) | -0.116 | -0.79 pp | Moindre familiarité avec les marchés étrangers |
| **EMPL_11** (Apprenti) | -0.115 | -0.78 pp | Formation en cours, mobilité limitée |

---

### EFFET NON-LINÉAIRE DE L'ÂGE

L'inclusion conjointe de AGEREV (+0.737) et AGEREV² (-0.751) révèle un **effet en U inversé** :

$$P(\text{transfrontalier}) \propto 0.737 \times \text{âge} - 0.751 \times \text{âge}^2$$

```
Probabilité transfrontalier
        │
        │       ╭─────────╮
        │      ╱           ╲
        │     ╱             ╲
        │    ╱               ╲
        │───╱─────────────────╲───
        └────────────────────────── Âge
           25     40     55
```

**Interprétation** : La probabilité d'être transfrontalier croît avec l'âge (accumulation d'expérience), atteint un **maximum autour de 35-45 ans**, puis décroît (moindre mobilité, ancrage familial, proximité retraite).

---

### SPÉCIFICATION FINALE POUR LES MODÈLES PROBIT

```
Y_transfrontalier ~ DEPT + AGEREV + AGEREV² + NA5 + EMPL + DIPL 
                  + STOCD + VOIT + SANI + NENFR + GS + DNAI 
                  + TYPL + INATC + ETUD + SEXE + TP + COUPLE
```

**17 variables conceptuelles → 81 features après one-hot encoding**

#### Points d'attention pour l'estimation :

1. **Multicolinéarité** : Résolue par les arbitrages (VIF < 5 pour toutes les variables conservées)
2. **Effet quadratique** : Conserver AGEREV² dans toutes les spécifications
3. **Variable COUPLE** : Importance faible (0.011) mais centrale pour le probit bivarié (Partie 5)
4. **Catégories de référence** : À définir explicitement lors de l'estimation probit

---

### FICHIERS GÉNÉRÉS

#### Matrices de données

| Fichier | Contenu | Usage |
|---------|---------|-------|
| X_raw_colinearity.csv | Features brutes (catégorielles texte) | Analyse de colinéarité |
| X_ml_ready.csv | Features encodées (one-hot) | Elastic Net / LASSO |
| X_final_clean.csv | Matrice finale après nettoyage | Modélisation |
| y_final.csv | Variable cible (TRANSFRONTALIER) | Modélisation |
| weights_final.csv | Poids d'enquête (IPONDI) | Pondération optionnelle |

#### Métadonnées et traçabilité

| Fichier | Contenu |
|---------|---------|
| dropped_columns.csv | Variables exclues avec raisons |
| variable_types.json | Classification numériques / catégorielles |
| categorical_info.json | Détail des modalités par variable |
| column_mapping.json | Correspondance variables brutes → dummies |
| rapport_arbitrages.txt | Documentation des décisions |
| elastic_net_results.json | Hyperparamètres et métriques |
| coefficients.csv | Coefficients Elastic Net par feature |

---

### SYNTHÈSE MÉTHODOLOGIQUE

#### Pourquoi l'Elastic Net ?

| Critère | LASSO | Ridge | Elastic Net |
|---------|:-----:|:-----:|:-----------:|
| Sélection de variables | ✅ | ❌ | ✅ |
| Gestion des corrélations | ❌ | ✅ | ✅ |
| Stabilité des groupes | ❌ | ✅ | ✅ |
| **Choix** | — | — | **✅** |

L'Elastic Net combine les avantages des deux approches : parcimonie (L1) et stabilité en présence de corrélations résiduelles (L2).

#### Performance obtenue

| Métrique | Valeur | Interprétation |
|----------|:------:|----------------|
| **AUC-ROC** | 0.8448 | Excellente — dans 85% des cas, le modèle attribue un score plus élevé à un transfrontalier qu'à un non-transfrontalier |
| Variables retenues | 81/81 | 100% — toutes les variables apportent de l'information après arbitrages |
| Cohérence économique | ✅ | Signes conformes à la théorie |

---

### LIMITES IDENTIFIÉES

| Limite | Impact | Atténuation |
|--------|--------|-------------|
| **Recall faible au seuil 0.5** | 7.2% des transfrontaliers détectés | Non pertinent pour l'objectif économétrique (pas de prédiction opérationnelle) |
| **Aucune variable éliminée** | Parcimonie limitée | Arbitrages préalables ont déjà réduit l'espace |
| **Corrélations résiduelles** | EMPL ↔ GS (ρ ≈ 0.5) | Géré par la composante Ridge (l1_ratio = 0.7) |

---

### TRANSITION VERS LA PARTIE 3

L'étape de machine learning a rempli son rôle : **identifier de manière objective et reproductible les variables pertinentes**. L'AUC de 0.8448 atteste du pouvoir discriminant élevé, et la cohérence économique des coefficients renforce la confiance dans la spécification.

La partie suivante mobilisera ces variables dans le cadre des **modèles probit**, qui permettront :
1. Une **interprétation structurelle** des effets via les coefficients et effets marginaux
2. L'analyse des **hétérogénéités de genre** par l'introduction d'interactions
3. La modélisation de la **décision conjointe au sein des couples** via le probit bivarié

**Le passage du machine learning à l'économétrie structurelle marque la transition d'une logique de prédiction vers une logique d'inférence causale et d'interprétation économique.**

---

### PIPELINE COMPLET

```
Base initiale (494 483 × 106)
    ↓
ÉTAPE 1 : Création variables dérivées (TRANSFRONTALIER, WORK_IN_PAYS)
    ↓
ÉTAPE 2 : Exclusion 25 variables (leakage, identifiants, constantes)
    ↓
ÉTAPE 3 : Analyse multicolinéarité (Spearman, Cramér, VIF, CAH)
    • 17 paires |ρ| ≥ 0.70
    • 665 paires V ≥ 0.50
    • 11 variables VIF > 10
    ↓
ÉTAPE 4 : Arbitrages économiques (7 paniers)
    • Panier 1 : GS vs STAT_GSEC → GS
    • Panier 2 : Variables âge → AGEREV + AGEREV²
    • Panier 3 : EMPL vs STAT/STATR → EMPL
    • Panier 4 : INATC vs IMMI/NAIDT → INATC
    • Panier 5 : COUPLE vs STAT_CONJ → COUPLE
    • Panier 6 : Structure familiale → NENFR
    • Panier 7 : NA5 vs NA17 → NA5
    ↓
ÉTAPE 5 : Nettoyage redondances résiduelles
    • Suppression TAF, HLML, STATR
    • 91 → 81 colonnes
    ↓
ÉTAPE 6 : Elastic Net (Grid Search CV)
    • C = 1.0, l1_ratio = 0.7
    • AUC-ROC = 0.8448
    • 81/81 features retenues
    ↓
MATRICE FINALE : 494 483 obs × 81 features
    → Prête pour estimation Probit (Parties 3-4-5)
```

---

*Partie 2 — Projet INSEE Mobilité Transfrontalière Grand Est*  
*Master 2 Statistique, Économétrie & Data Science — Université de Strasbourg*

# PARTIE 3 : MODÈLE PROBIT BINAIRE — DÉTERMINANTS DU TRAVAIL TRANSFRONTALIER

## 1. CONTEXTE ET OBJECTIF

### 1.1 Positionnement dans le projet

| Étape | Modèle | Objectif | Statut |
|-------|--------|----------|--------|
| **Partie 1** | — | Préparation et validation de la base de données | ✅ Terminé |
| **Partie 2** | Elastic Net | Sélection des variables pertinentes | ✅ Terminé |
| **Partie 3** | **Probit binaire** | **Estimation des déterminants individuels** | ✅ Cette section |


### 1.2 Objectif de cette partie

Estimer un **modèle probit binaire** pour identifier les facteurs individuels qui influencent la probabilité d'exercer une activité professionnelle hors de France (Luxembourg, Allemagne, Suisse, Belgique).

**Variable dépendante** :
$$Y_i = \begin{cases} 1 & \text{si l'individu } i \text{ est travailleur transfrontalier} \\ 0 & \text{sinon} \end{cases}$$

**Modèle estimé** :
$$P(Y_i = 1 \mid X_i) = \Phi(X_i' \beta)$$

où $\Phi(\cdot)$ désigne la fonction de répartition de la loi normale centrée réduite.

### 1.3 Données utilisées

| Caractéristique | Valeur |
|-----------------|--------|
| Observations | 494 483 |
| Variables explicatives | 65 (après exclusion références) |
| Transfrontaliers (Y = 1) | 44 264 (8,95 %) |
| Non-transfrontaliers (Y = 0) | 450 219 (91,05 %) |

---

## 2. SPÉCIFICATION DU MODÈLE

### 2.1 Formulation mathématique

Le modèle probit latent s'écrit :

$$Y^*_i = \beta_0 + \beta_1 \text{AGEREV}_i + \beta_2 \text{AGEREV}^2_i + \sum_k \gamma_k D_{ki} + \varepsilon_i, \quad \varepsilon_i \sim \mathcal{N}(0,1)$$

avec $Y_i = \mathbb{1}_{Y^*_i > 0}$ et où $D_{ki}$ désigne les indicatrices des variables catégorielles.

### 2.2 Variables explicatives retenues (17 blocs)

| Catégorie | Variable | Description | Modalités | Référence |
|-----------|----------|-------------|-----------|-----------|
| **Démographie** | AGEREV | Âge révolu (+ terme quadratique) | Continue | — |
| | SEXE | Genre | 2 | Homme |
| **Géographie** | DEPT | Département de résidence | 10 | Marne (51) |
| | DNAI | Lieu de naissance | 3 | Né Grand Est |
| **Capital humain** | DIPL | Diplôme le plus élevé | 12 | CAP/BEP |
| | ETUD | Inscription aux études | 2 | Non inscrit |
| **Emploi** | GS | Groupe socioprofessionnel | 6 | Employés |
| | EMPL | Condition d'emploi | 9 | CDI/Titulaire |
| | NA5 | Secteur d'activité économique | 5 | Services |
| | TP | Temps de travail | 2 | Temps complet |
| **Famille** | COUPLE | Vie en couple | 2 | En couple |
| | NENFR | Nombre d'enfants dans le ménage | 6 | 0 enfant |
| **Logement** | STOCD | Statut d'occupation du logement | 5 | Propriétaire |
| | TYPL | Type de logement | 6 | Maison |
| | SANI | Installations sanitaires | 4 | Standard |
| **Mobilité** | VOIT | Nombre de voitures du ménage | 4 | 1 voiture |
| **Origine** | INATC | Nationalité | 2 | Français |

**Total** : 17 variables conceptuelles → **65 coefficients estimés** (hors constante)

### 2.3 Choix des catégories de référence

Deux stratégies combinées :
1. **Modalité la plus fréquente** (convention standard) : GS_5 (Employés), EMPL_16 (CDI), etc.
2. **Choix économiquement interprétable** :
   - **DEPT_51 (Marne)** : département intérieur non frontalier → tous les effets frontaliers en positif
   - **NENFR_0** : 0 enfant → effets de la parentalité directement lisibles
   - **VOIT_1** : 1 voiture → référence médiane

---

## 3. RÉSULTATS DE L'ESTIMATION

### 3.1 Qualité de l'ajustement global

| Indicateur | Valeur | Interprétation |
|------------|:------:|----------------|
| **Nombre d'observations** | 494 483 | Échantillon très large |
| **Pseudo R² (McFadden)** | 0,2393 | Bon ajustement pour modèle binaire |
| **Log-vraisemblance** | -113 380,76 | — |
| **Log-vraisemblance nulle** | -149 050 | — |
| **AIC** | 226 893,52 | — |
| **BIC** | 227 626,87 | — |
| **Test LR (χ²)** | 71 340 | — |
| **p-value LR** | < 0,001 | Modèle globalement très significatif |
| **Convergence** | Oui (10 itérations) | Algorithme stable |

### 3.2 Significativité des variables

| Catégorie | Nombre |
|-----------|:------:|
| Variables significatives au seuil 1% | 56 / 65 |
| Variables significatives au seuil 5% | 2 / 65 |
| Variables significatives au seuil 10% | 2 / 65 |
| Variables non significatives | 5 / 65 |

**Variables non significatives** : GS_1 (agriculteurs), DEPT_10 (Aube), DEPT_52 (Haute-Marne), NA5_BE (industrie), TYPL_5 (habitation de fortune).

---

## 4. EFFETS MARGINAUX MOYENS (AME)

### 4.1 Convention de lecture

Les résultats sont présentés en **effets marginaux moyens (AME)**, exprimés en **points de pourcentage (pp)**.

| Méthode | Formule | Usage |
|---------|---------|-------|
| **MEM** | $\phi(\bar{X}'\beta) \times \beta$ | Non utilisé (individu fictif) |
| **AME** | $\frac{1}{n} \sum_i \phi(X_i'\beta) \times \beta$ | **Reporté** (effet moyen réel) |

**Ratio MEM/AME** : Les AME sont environ **2× supérieurs** aux MEM (corrélation = 1,00), reflétant la concentration géographique du phénomène.

### 4.2 Hiérarchie des effets (Top 12 positifs et négatifs)

**TOP 6 EFFETS POSITIFS** :

| Variable | AME (pp) | Interprétation |
|----------|:--------:|----------------|
| DEPT_57 (Moselle) | **+30,63** | Proximité Luxembourg |
| DEPT_68 (Haut-Rhin) | **+26,28** | Proximité Suisse/Allemagne |
| DEPT_54 (M.-et-Moselle) | **+24,39** | Proximité Luxembourg |
| DEPT_55 (Meuse) | **+19,15** | Proximité Luxembourg/Belgique |
| DEPT_67 (Bas-Rhin) | **+18,51** | Proximité Allemagne |
| INATC_2 (Étranger) | **+6,22** | Réseaux transnationaux |

**TOP 6 EFFETS NÉGATIFS** :

| Variable | AME (pp) | Interprétation |
|----------|:--------:|----------------|
| SANI_X (Hors logement) | **-19,57** | Situation résidentielle précaire |
| NA5_AZ (Agriculture) | **-12,00** | Activité terrienne non délocalisable |
| EMPL_13 (Emplois aidés) | **-9,38** | Dispositifs nationaux |
| NA5_OQ (Administration) | **-8,30** | Secteur public français |
| TYPL_6 (Logement foyer) | **-7,94** | Situation précaire |
| EMPL_21 (Indépendant) | **-7,15** | Activité ancrée localement |

---

## 5. INTERPRÉTATION ÉCONOMIQUE PAR THÈME

### 5.1 Effets géographiques (DEPT)

> **Statut** : Variable exogène — le département de résidence précède logiquement le choix professionnel.

**Catégorie de référence** : Marne (DEPT_51) — département intérieur

| Département | AME | Interprétation |
|-------------|:---:|----------------|
| Moselle (57) | +30,6 pp | 1er bassin transfrontalier (Luxembourg) |
| Haut-Rhin (68) | +26,3 pp | Proximité Suisse/Allemagne (Bâle) |
| Meurthe-et-Moselle (54) | +24,4 pp | Accès bassin luxembourgeois |
| Meuse (55) | +19,1 pp | Proximité Luxembourg/Belgique |
| Bas-Rhin (67) | +18,5 pp | Proximité Allemagne (Kehl) |
| Vosges (88) | +6,7 pp | Relief montagneux, accessibilité limitée |
| Aube (10) | NS | Éloigné des frontières |
| Haute-Marne (52) | NS | Éloigné des frontières |

**Conclusion** : La géographie constitue le **premier déterminant** du travail transfrontalier, avec des effets considérables sans commune mesure avec les autres variables.

### 5.2 Réseaux transnationaux (INATC, DNAI)

> **Statut** : Variables exogènes — nationalité et lieu de naissance sont antérieurs à l'emploi.

| Variable | AME | Interprétation |
|----------|:---:|----------------|
| Nationalité étrangère (INATC_2) | +6,22 pp | Capital linguistique, réseaux |
| Né à l'étranger (DNAI_NEETRANGER) | +4,39 pp | Familiarité avec l'international |
| Né ailleurs en France (DNAI_NEAUTREFR) | -0,38 pp | Moindre ancrage frontalier |

**Effet cumulé** : Un individu de nationalité étrangère né à l'étranger cumule +10,6 pp par rapport à un Français né dans le Grand Est.

### 5.3 Profil démographique (AGEREV, SEXE)

> **Statut** : Variables exogènes — l'âge et le sexe sont des caractéristiques individuelles prédéterminées.

**Effet de l'âge** (forme quadratique) :

$$\frac{\partial P}{\partial \text{âge}} \propto 0,0326 - 2 \times 0,0004 \times \text{âge}$$

| Indicateur | Valeur |
|------------|:------:|
| Coefficient linéaire | +0,41 pp/an |
| Coefficient quadratique | -0,005 pp/an² |
| **Âge optimal** | **≈ 41 ans** |

**Interprétation** : Forme en **U inversé** — croissance jusqu'à 41 ans (accumulation d'expérience), puis décroissance (ancrage familial, pénibilité des trajets).

**Effet du genre** :

| Variable | AME |
|----------|:---:|
| Femme (SEXE_2) | -1,58 pp |

### 5.4 Situation familiale (NENFR, COUPLE)

> **Statut** : Variables probablement exogènes — la structure familiale influence les choix professionnels.

**Effet du nombre d'enfants** (référence : 0 enfant) :

| Enfants | AME | Interprétation |
|---------|:---:|----------------|
| 1 enfant | -0,89 pp | Légère pénalité |
| 2 enfants | -1,18 pp | Pénalité croissante |
| 3 enfants | -2,36 pp | Pénalité marquée |
| 4+ enfants | -2,67 pp | Pénalité maximale |
| Hors famille (Z) | +1,00 pp | Départ des enfants libère mobilité |

**Effet gradient** : La pénalité croît avec le nombre d'enfants, avec un seuil critique autour de 3 enfants.

**Effet du couple** :

| Variable | AME |
|----------|:---:|
| Pas en couple (COUPLE_2) | -1,81 pp |

### 5.5 Caractéristiques professionnelles (GS, NA5, EMPL)

#### Catégorie socioprofessionnelle (GS)

> **Statut** : Variable probablement exogène — la CSP structure l'accès au marché transfrontalier.

**Catégorie de référence** : Employés (GS_5)

| CSP | AME | Interprétation |
|-----|:---:|----------------|
| **Ouvriers (GS_6)** | **+1,17 pp** | Industrie transfrontalière |
| Professions intermédiaires (GS_4) | -0,50 pp | Souvent secteur public |
| Cadres (GS_3) | -1,06 pp | Opportunités locales satisfaisantes |
| **Artisans, commerçants (GS_2)** | **-3,87 pp** | Activité ancrée localement |
| Agriculteurs (GS_1) | -1,35 pp (NS) | Activité terrienne |

**Double marché** : Les ouvriers (industrie) et les cadres très diplômés (finance, conseil) sont favorisés.

#### Secteur d'activité (NA5)

**Catégorie de référence** : Commerce, transports et services divers (NA5_GU)

| Secteur | AME | Interprétation |
|---------|:---:|----------------|
| Construction (NA5_FZ) | -0,26 pp (NS) | Effet neutre |
| Industrie (NA5_BE) | -0,26 pp (NS) | Effet neutre |
| **Administration, enseignement, santé (NA5_OQ)** | **-8,30 pp** | Emplois publics non transfrontaliers |
| **Agriculture (NA5_AZ)** | **-12,00 pp** | Activité terrienne par nature |

#### Condition d'emploi (EMPL)

**Catégorie de référence** : CDI/Titulaire fonction publique (EMPL_16)

| Type de contrat | AME | Interprétation |
|-----------------|:---:|----------------|
| **Stagiaires rémunérés (EMPL_14)** | **+5,21 pp** | Stage transfrontalier |
| Intérim (EMPL_12) | -0,97 pp | Agences spécialisées |
| CDD (EMPL_15) | -1,26 pp | Formes précaires exclues |
| Employeurs (EMPL_22) | -5,95 pp | Entrepreneuriat local |
| **Apprentissage (EMPL_11)** | **-6,59 pp** | CFA locaux |
| Indépendants (EMPL_21) | -7,15 pp | Activité ancrée |
| **Emplois aidés (EMPL_13)** | **-9,38 pp** | Dispositifs nationaux |

### 5.6 Capital humain (DIPL, ETUD)

> **Statut** : Variable exogène — le diplôme est obtenu avant l'entrée sur le marché du travail.

**Catégorie de référence** : CAP, BEP (DIPL_13)

| Niveau de diplôme | AME | Interprétation |
|-------------------|:---:|----------------|
| **Grande école (DIPL_19)** | **+3,07 pp** | Prime aux très diplômés |
| **Bac+5 Master (DIPL_18)** | **+2,79 pp** | Finance, conseil, institutions UE |
| Bac+4 (DIPL_11) | +1,54 pp | Effet positif |
| Bac pro (DIPL_17) | +1,00 pp | Qualifications professionnalisantes |
| Bac général (DIPL_14) | -0,63 pp | Effet neutre |
| Bac+2 BTS/DUT (DIPL_16) | -0,61 pp | Débouchés locaux |
| Sans diplôme (DIPL_3) | -1,21 pp | Barrières multiples |
| Sans diplôme primaire (DIPL_2) | -3,19 pp | Exclusion |
| **Doctorat (DIPL_1)** | **-6,55 pp** | Recherche publique non transfrontalière |

**Relation non monotone** : Les très diplômés (Bac+5, grandes écoles) et les doctorats présentent des effets opposés, reflétant la différence entre secteur privé international et recherche publique.

### 5.7 Variables potentiellement endogènes (STOCD, VOIT, TYPL, TP)

> **Statut** : **Risque élevé de causalité inverse** — les revenus frontaliers peuvent permettre l'accession à la propriété et la multi-motorisation. Interprétation en termes d'**associations**, non de causalité.

#### Motorisation (VOIT)

| Voitures | AME |
|----------|:---:|
| 0 voiture | **-4,06 pp** |
| 2 voitures | +1,48 pp |
| 3+ voitures | +2,09 pp |

#### Statut d'occupation (STOCD)

| Statut | AME |
|--------|:---:|
| Locataire privé (STOCD_21) | -1,01 pp |
| Locataire meublé (STOCD_23) | -2,61 pp |
| Logé gratuit (STOCD_30) | -3,57 pp |
| **Locataire HLM (STOCD_22)** | **-6,21 pp** |

#### Type de logement (TYPL)

| Type | AME |
|------|:---:|
| Appartement (TYPL_2) | -2,09 pp |
| Logement foyer (TYPL_6) | -7,94 pp |
| Habitation fortune (TYPL_5) | -2,21 pp (NS) |

#### Temps de travail (TP)

| Type | AME |
|------|:---:|
| Temps partiel (TP_2) | +1,47 pp |

---

## 6. TESTS DE ROBUSTESSE

### 6.1 Batterie de tests réalisés

| # | Test | Objectif | Résultat |
|---|------|----------|----------|
| 1 | Probit de base | Convergence, qualité globale | ✅ Convergence en 10 itérations |
| 2 | Link Test (Pregibon) | Erreur de spécification | ⚠️ Rejeté (p < 0,001) |
| 3 | Hosmer-Lemeshow | Ajustement du modèle | ⚠️ Rejeté (χ² = 304,5) |
| 4 | Probit vs Logit | Robustesse au choix du lien | ✅ Corrélation MFX = 0,995 |
| 5 | Exclusion variables endogènes | Sensibilité | ✅ Variation < 5% |
| 6 | Sous-échantillons | Stabilité par groupes | ✅ Pseudo R² stables |
| 7 | Analyse de sensibilité | Observations extrêmes | ✅ Caractérisées |
| 8 | Forme fonctionnelle âge | Spécification quadratique | ✅ Test LR significatif |

### 6.2 Interprétation des tests HL et Link rejetés

Les tests classiques sont rejetés avec des p-values nulles **en raison de la taille de l'échantillon** (n = 494 000). À cette échelle, ces tests disposent d'une puissance statistique telle qu'ils détectent la moindre déviation, aussi infime soit-elle.

**Solution** : Remplacer par des diagnostics adaptés aux grands échantillons.

### 6.3 Équivalence Probit/Logit

| Critère | Probit | Logit |
|---------|:------:|:-----:|
| Log-vraisemblance | -113 380,76 | -113 285,43 |
| Pseudo R² | 0,2393 | 0,2399 |
| AIC | 226 893,52 | 226 702,86 |
| BIC | 227 626,87 | 227 436,20 |
| **Corrélation effets marginaux** | **0,9953** | |

**Conclusion** : Les deux spécifications sont quasi-équivalentes. Le choix du probit est robuste.

### 6.4 Robustesse à l'endogénéité

**Variables exclues** : 15 (VOIT, STOCD, TYPL, SANI)

| Modèle | Log-L | Pseudo R² | Nb vars |
|--------|:-----:|:---------:|:-------:|
| Complet | -113 380,76 | 0,2393 | 66 |
| Réduit | -116 153 | 0,2207 | 51 |

**Stabilité des coefficients clés** :

| Variable | Complet | Réduit | Variation |
|----------|:-------:|:------:|:---------:|
| DEPT_57 | 2,4367 | 2,4566 | +0,8% ✅ |
| DEPT_68 | 2,0908 | 2,1195 | +1,4% ✅ |
| SEXE_2 | -0,1255 | -0,1310 | +4,4% ✅ |
| AGEREV | 0,0326 | 0,0322 | -1,4% ✅ |

**Conclusion** : Les coefficients des variables exogènes sont stables, validant les conclusions principales.

---

## 7. DIAGNOSTICS ADAPTÉS AUX GRANDS ÉCHANTILLONS

### 7.1 Score de Brier

| Métrique | Valeur | Interprétation |
|----------|:------:|----------------|
| Brier Score | 0,0681 | Proche de 0 = bon |
| Brier (modèle nul) | 0,0815 | Prédiction naïve P̄ |
| **Brier Skill Score** | **0,1643** | Amélioration de 16,4% vs nul |

### 7.2 Calibration par déciles

| Décile | N | Prédit | Observé | Écart |
|:------:|:---:|:------:|:-------:|:-----:|
| 0 | 49 455 | 0,0159 | 0,0142 | -0,0017 |
| 1 | 49 437 | 0,0269 | 0,0265 | -0,0004 |
| 2 | 49 456 | 0,0354 | 0,0360 | +0,0006 |
| 3 | 49 458 | 0,0434 | 0,0435 | +0,0001 |
| 4 | 49 444 | 0,0526 | 0,0543 | +0,0017 |
| 5 | 49 445 | 0,0638 | 0,0633 | -0,0005 |
| 6 | 49 442 | 0,0784 | 0,0797 | +0,0013 |
| 7 | 49 452 | 0,1004 | 0,0992 | -0,0012 |
| 8 | 49 446 | 0,1433 | 0,1430 | -0,0003 |
| 9 | 49 448 | 0,3567 | 0,3561 | -0,0006 |

**Écart moyen absolu de calibration** : **0,0033** (0,33 pp)

✅ **Excellente calibration** — le modèle ne sur-estime ni ne sous-estime systématiquement les probabilités.

### 7.3 Discrimination (AUC-ROC)

| Métrique | Valeur |
|----------|:------:|
| AUC point estimate | **0,8486** |
| IC 95% Bootstrap | [0,8480 ; 0,8502] |

✅ **Excellente discrimination** (AUC > 0,8)

### 7.4 Validation croisée 5-fold

| Métrique | Moyenne | Écart-type |
|----------|:-------:|:----------:|
| AUC | 0,8410 | ± 0,0040 |
| Brier | 0,0692 | ± 0,0003 |

✅ **Très stable entre folds** (CV < 1%) — absence de sur-ajustement confirmée.

---

## 8. ANALYSE DES RÉSIDUS EXTRÊMES

### 8.1 Distribution des résidus

| Seuil | N observations | % |
|:-----:|:--------------:|:-:|
| \|résidu\| > 2σ | 36 005 | 7,28% |
| \|résidu\| > 2,5σ | 21 432 | 4,33% |
| \|résidu\| > 3σ | 21 415 | 4,33% |

### 8.2 Types de résidus extrêmes

| Type | N | Description |
|------|:-:|-------------|
| **Faux négatifs** | 19 332 | Transfrontaliers avec P < 20% |
| Faux positifs | 244 | Non-transfrontaliers avec P > 30% |

**Observation clé** : 99,3% des résidus extrêmes sont des **transfrontaliers non prédits**.

### 8.3 Répartition géographique

| Département | % d'extrêmes | Taux transf. |
|-------------|:------------:|:------------:|
| **Moselle (57)** | **15,5%** | 21,6% |
| **Haut-Rhin (68)** | **12,2%** | 14,1% |
| Meurthe-et-Moselle (54) | 11,6% | 9,7% |
| Bas-Rhin (67) | 9,8% | 4,6% |
| Marne (51) | 0,1% | 0,1% |
| Aube (10) | 0,1% | 0,1% |

**Conclusion** : Les résidus extrêmes sont concentrés dans les **départements frontaliers**. Ils reflètent l'hétérogénéité réelle des comportements dans les zones proches des frontières, et ne doivent pas être supprimés.

### 8.4 Impact sur le Pseudo R²

| Échantillon | N | Pseudo R² |
|-------------|:-:|:---------:|
| Complet | 494 483 | 0,2393 |
| Sans résidus > 2,5σ | 473 051 | **0,4804** |

Le quasi-doublement du Pseudo R² après exclusion confirme que ces individus constituent une **sous-population structurellement différente** dont le comportement échappe aux variables du modèle.

---

## 9. SYNTHÈSE DES DÉTERMINANTS

### 9.1 Classement par amplitude (AME)

| Rang | Variable | AME (pp) | Classification |
|:----:|----------|:--------:|----------------|
| 1 | DEPT_57 (Moselle) | +30,63 | Exogène |
| 2 | DEPT_68 (Haut-Rhin) | +26,28 | Exogène |
| 3 | DEPT_54 (M.-et-Moselle) | +24,39 | Exogène |
| 4 | DEPT_55 (Meuse) | +19,15 | Exogène |
| 5 | DEPT_67 (Bas-Rhin) | +18,51 | Exogène |
| 6 | NA5_AZ (Agriculture) | -12,00 | Prob. exogène |
| 7 | EMPL_13 (Emplois aidés) | -9,38 | Ambigu |
| 8 | NA5_OQ (Administration) | -8,30 | Prob. exogène |
| 9 | DEPT_88 (Vosges) | +6,68 | Exogène |
| 10 | DIPL_1 (Doctorat) | -6,55 | Exogène |
| 11 | STOCD_22 (HLM) | -6,21 | **Endogène** |
| 12 | INATC_2 (Étranger) | +6,22 | Exogène |

### 9.2 Classification des variables par risque d'endogénéité

| Statut | Variables |
|--------|-----------|
| **Exogène** | Département, âge, sexe, nationalité, lieu de naissance, diplôme |
| **Probablement exogène** | Nombre d'enfants, secteur d'activité, CSP |
| **Ambigu** | Type de contrat, temps de travail |
| **Potentiellement endogène** | Statut d'occupation, nombre de voitures, type de logement |

---

## 10. LIMITES MÉTHODOLOGIQUES

### 10.1 Variables omises

Le modèle n'inclut pas :
- La **distance précise à la frontière** (seul le département est disponible)
- Les **compétences linguistiques** (allemand, luxembourgeois)
- Le **différentiel salarial individuel** entre emploi local et frontalier
- Les **préférences individuelles** (aversion aux trajets, attachement territorial)

### 10.2 Hétérogénéité non observée

Des caractéristiques individuelles non mesurées (motivation, réseau professionnel, santé) peuvent influencer simultanément le choix de résidence et le travail frontalier.

### 10.3 Coupe transversale

L'analyse sur une seule année (2022) ne permet pas d'étudier les dynamiques temporelles (entrées/sorties du travail frontalier, effets de la conjoncture).

---

## 11. FICHIERS GÉNÉRÉS

| Fichier | Description |
|---------|-------------|
| `BDD_PROBIT.csv` | Base prête pour estimation (65 variables) |
| `probit_model1_summary.txt` | Résumé complet du modèle |
| `AME_results.csv` | Effets marginaux moyens |
| `coefficients_probit.csv` | Coefficients estimés |
| `rapport_robustesse.txt` | Tests de robustesse |
| `rapport_robustesse_v3.txt` | Diagnostics grands échantillons |

---

## 12. CONCLUSION

Cette analyse économétrique met en évidence une **hiérarchie claire des déterminants** du travail transfrontalier dans le Grand Est :

1. **La géographie domine** : Résider dans un département frontalier constitue le facteur le plus déterminant, avec des effets marginaux considérables (+30,6 pp pour la Moselle, +26,3 pp pour le Haut-Rhin, +24,4 pp pour la Meurthe-et-Moselle).

2. **Les réseaux transnationaux constituent le deuxième facteur** : La nationalité étrangère (+6,2 pp) et la naissance à l'étranger (+4,4 pp) facilitent significativement l'accès au travail frontalier.

3. **Le profil socioprofessionnel sélectionne** : Le travail frontalier concerne prioritairement les ouvriers qualifiés (+1,2 pp) et les cadres très diplômés (Bac+5 : +2,8 pp), tout en excluant structurellement les artisans (-3,9 pp), le secteur public (-8,3 pp) et l'agriculture (-12,0 pp).

4. **La famille contraint de manière non linéaire** : La présence d'enfants réduit la probabilité selon un gradient croissant (-0,9 pp pour 1 enfant, -2,4 pp pour 3 enfants, -2,7 pp pour 4+).

5. **Le genre structure les comportements** : Les femmes présentent une probabilité inférieure de 1,6 pp, toutes choses égales par ailleurs.

6. **L'âge suit une trajectoire en U inversé** : Pic estimé à 41 ans, reflétant l'arbitrage entre accumulation d'expérience et ancrage territorial.

7. **Le modèle est statistiquement robuste** : Pseudo R² de 0,239, AUC de 0,849, calibration quasi-parfaite. La validation croisée (AUC = 0,841 ± 0,004) confirme l'absence de sur-ajustement.

---

## 13. TRANSITION VERS PARTIE 4

Les résultats fournissent une base solide pour les extensions :

- **Partie 4 (Probit avec interactions de genre)** : Tester si les déterminants — notamment la parentalité, le temps de travail et le diplôme — affectent différemment les hommes et les femmes.

- **Partie 5 (Probit bivarié)** : Modéliser la coordination des décisions professionnelles au sein des couples.


# PARTIE 4 : MODÈLE PROBIT AVEC INTERACTIONS PAR GENRE — HÉTÉROGÉNÉITÉ DES EFFETS

## OBJECTIF ET POSITIONNEMENT

Tester l'**hétérogénéité des effets selon le genre** en introduisant des termes d'interaction entre la variable SEXE et les covariables sélectionnées. L'objectif est de déterminer si les déterminants du travail transfrontalier — notamment la parentalité, le temps de travail, le diplôme — affectent différemment les hommes et les femmes.

**Question centrale** : Les facteurs qui favorisent ou freinent la mobilité transfrontalière agissent-ils de la même manière pour les deux genres ?

**Transition depuis la Partie 3** : Le modèle probit simple estimé en Partie 3 a révélé un coefficient de SEXE_2 (Femme) indiquant une probabilité inférieure de ~1.6 pp d'être travailleur transfrontalier. Cependant, ce coefficient unique suppose un effet homogène du genre et ne capture pas les mécanismes différenciés.

**Fichier source** : BDD_PROBIT.csv (X) + y_final.csv (Y)

**Notebook** : /mnt/project/4_Mode_le_Probit_avec_interaction_genre.ipynb (5 881 lignes)

---

## SPÉCIFICATION DU MODÈLE

### Formulation mathématique

$$P(Y_i = 1) = \Phi\left(\beta_0 + \beta_1 \cdot \text{Femme}_i + \beta_2 \cdot X_i + \gamma \cdot (\text{Femme}_i \times X_i) + \delta \cdot Z_i\right)$$

où :
- $Y_i$ : statut de travailleur transfrontalier (1 = oui, 0 = non)
- $\text{Femme}_i$ : indicatrice du genre (SEXE_2)
- $X_i$ : variable pour laquelle on teste l'interaction
- $Z_i$ : autres covariables de contrôle
- $\Phi(\cdot)$ : fonction de répartition de la loi normale

### Interprétation des coefficients

| Coefficient | Interprétation |
|-------------|----------------|
| $\beta_2$ | Effet de $X$ pour les **hommes** (référence) |
| $\gamma$ | Effet **différentiel** pour les femmes |
| $\beta_2 + \gamma$ | Effet **total** de $X$ pour les **femmes** |

Le test de significativité de $\gamma$ permet de conclure :
- $\gamma \neq 0$ significatif → l'effet de $X$ **diffère** selon le genre
- $\gamma = 0$ → l'effet de $X$ est **homogène** entre hommes et femmes

---

## INTERACTIONS RETENUES

**10 dimensions** testées → **45 termes d'interaction** créés :

| # | Variable | Dimension | Hypothèse économique | N interactions |
|---|----------|-----------|----------------------|:--------------:|
| 1 | **NENFR** | Parentalité | Pénalité parentale plus forte pour les femmes | 5 |
| 2 | **TP** | Temps de travail | Le temps partiel comme contrainte genrée | 1 |
| 3 | **DIPL** | Capital humain | Rendement différencié du diplôme | 11 |
| 4 | **GS** | Position socioprofessionnelle | Effet de la PCS selon le genre | 5 |
| 5 | **NA5** | Secteur d'activité | Segmentation sectorielle genrée | 4 |
| 6 | **EMPL** | Conditions d'emploi | CDI/indépendance et genre | 8 |
| 7 | **AGEREV** | Cycle de vie | Profil âge-emploi différencié H/F | 2 |
| 8 | **STOCD** | Ancrage résidentiel | Propriété et mobilité genrée | 4 |
| 9 | **DNAI** | Intersectionnalité | Genre × origine géographique | 2 |
| 10 | **VOIT** | Accès à la mobilité | Accès genré au véhicule du ménage | 3 |

**Total** : 65 variables originales + 45 interactions = **110 variables explicatives**

---

## DONNÉES UTILISÉES

| Caractéristique | Valeur |
|-----------------|--------|
| Observations | 494 483 |
| Variables originales | 65 |
| Interactions créées | 45 |
| **Variables totales** | **110** |
| Transfrontaliers (Y = 1) | 44 264 (8.95%) |
| Non-transfrontaliers (Y = 0) | 450 219 (91.05%) |
| Hommes | 256 784 (51.9%) |
| Femmes | 237 699 (48.1%) |

---

## DIAGNOSTIC DE MULTICOLINÉARITÉ (VIF)

### Résultats avant correction

| Métrique | Valeur |
|----------|:------:|
| VIF moyen | 14.28 |
| VIF médian | 2.51 |
| VIF maximum | 678.92 |
| Variables VIF < 5 | 104 (94.5%) |
| Variables VIF 5-10 | 1 (0.9%) |
| Variables VIF ≥ 10 | 5 (4.5%) |

### Variables problématiques identifiées

| Variable | VIF | Source du problème |
|----------|:---:|-------------------|
| SEXE2_x_AGEREV | 678.92 | Interaction âge × genre |
| SEXE2_x_AGEREV_sq | 261.08 | Interaction âge² × genre |
| SEXE_2 | 152.86 | Variable présente dans 45 interactions |
| AGEREV | 101.10 | Corrélation avec AGEREV² |
| AGEREV_sq | 97.35 | Corrélation avec AGEREV |

### Solution appliquée : Centrage de l'âge sur l'optimum

Transformation : $\text{AGEREV\_c} = \text{AGEREV} - 41$

Cette transformation :
1. Réduit drastiquement la corrélation entre termes linéaire et quadratique
2. Enrichit l'interprétation : le coefficient mesure l'effet d'un écart par rapport à l'âge optimal (41 ans)
3. Préserve l'invariance des prédictions et effets marginaux

### Résultats après correction

| Variable | VIF avant | VIF après | Réduction |
|----------|:---------:|:---------:|:---------:|
| AGEREV_c | 101.1 | 2.9 | -97% |
| AGEREV_c_sq | 97.3 | 2.6 | -97% |
| SEXE_2 | 152.9 | 20.1 | -87% |
| SEXE2_x_AGEREV_c | 678.9 | 2.8 | -100% |
| SEXE2_x_AGEREV_c_sq | 261.1 | 3.5 | -99% |

| Métrique | Avant | Après | Amélioration |
|----------|:-----:|:-----:|:------------:|
| VIF moyen | 14.28 | 2.83 | -80% |
| VIF maximum | 678.92 | 20.13 | -97% |
| Variables VIF ≥ 10 | 5 | 1 | -80% |

**Conclusion** : Seule SEXE_2 conserve un VIF > 10 (structurellement inévitable car présente dans 45 interactions). Cela n'affecte pas l'interprétation des effets différentiels.

---

## RÉSULTATS D'ESTIMATION

### Qualité de l'ajustement

| Métrique | Valeur | Interprétation |
|----------|:------:|----------------|
| **Pseudo-R² (McFadden)** | 0.2424 | Excellent, supérieur à la Partie 3 |
| Log-vraisemblance (modèle) | -112 913.73 | — |
| Log-vraisemblance (null) | -149 045.06 | — |
| LR statistic | 72 262.66 | Modèle globalement significatif |
| LR p-value | < 0.001 | — |
| **AIC** | 226 049.45 | Inférieur à la Partie 3 |
| **BIC** | 227 282.80 | Inférieur à la Partie 3 |
| Paramètres estimés | 111 | 110 variables + constante |
| Convergence | ✅ Atteinte en 10 itérations | 33 secondes |

### Significativité des interactions

| Catégorie | Nombre | Pourcentage |
|-----------|:------:|:-----------:|
| **Interactions significatives (p < 0.05)** | **21** | **47%** |
| Interactions non significatives | 24 | 53% |
| **Total** | **45** | 100% |

---

## TESTS DE ROBUSTESSE

### Test de Wald global (toutes interactions)

| Métrique | Valeur |
|----------|:------:|
| Statistique de Wald | 926.52 |
| Degrés de liberté | 45 |
| P-value | 5.86 × 10⁻¹⁶⁵ |

**Conclusion** : Rejet de H₀ au seuil 0.1% — Les interactions sont **conjointement très significatives**.

### Test du rapport de vraisemblance (LR)

Comparaison : Modèle avec interactions vs Modèle sans interactions (Partie 3)

| Métrique | Sans interactions | Avec interactions |
|----------|:-----------------:|:-----------------:|
| Log-vraisemblance | -113 380.76 | -112 913.73 |
| Paramètres | 66 | 111 |

| Test LR | Valeur |
|---------|:------:|
| Statistique LR | 934.07 |
| Degrés de liberté | 45 |
| Seuil χ²(45) à 99% | 69.96 |
| P-value | ≈ 0 |

**Conclusion** : Rejet de H₀ au seuil 0.1% — Le modèle avec interactions est **significativement meilleur**.

### Tests de Wald par groupe thématique

| Dimension | Wald | ddl | P-value | Conclusion |
|-----------|:----:|:---:|:-------:|:----------:|
| **Temps partiel (TP)** | **129.82** | 1 | < 0.001 | ✅ Validé |
| **Intersectionnalité (DNAI)** | **202.33** | 2 | < 0.001 | ✅ Validé |
| **Secteur d'activité (NA5)** | **212.12** | 4 | < 0.001 | ✅ Validé |
| **Cycle de vie (AGEREV)** | **158.16** | 2 | < 0.001 | ✅ Validé |
| **Conditions d'emploi (EMPL)** | **76.95** | 8 | < 0.001 | ✅ Validé |
| **Parentalité (NENFR)** | **74.26** | 5 | < 0.001 | ✅ Validé |
| **Position sociale (GS)** | **55.64** | 5 | < 0.001 | ✅ Validé |
| **Capital humain (DIPL)** | **33.62** | 11 | < 0.001 | ✅ Validé |
| **Accès à la mobilité (VOIT)** | **16.25** | 3 | 0.001 | ✅ Validé |
| Ancrage résidentiel (STOCD) | 5.54 | 4 | 0.236 | ❌ Non validé |

**Résultat** : **9 groupes sur 10** sont conjointement significatifs au seuil de 5%.

### Comparaison des métriques

| Métrique | Sans interactions | Avec interactions | Variation |
|----------|:-----------------:|:-----------------:|:---------:|
| Pseudo R² | 0.2393 | 0.2424 | +1.31% |
| Log-vraisemblance | -113 381 | -112 914 | +467 |
| AIC | 226 894 | 226 049 | ✓ Mieux |
| BIC | 227 627 | 227 283 | ✓ Mieux |
| Paramètres | 66 | 111 | +45 |

**Interprétation** : Les critères AIC et BIC, qui pénalisent la complexité, sont améliorés malgré 45 paramètres supplémentaires. L'apport informatif des interactions justifie leur inclusion.

---

## EFFETS MARGINAUX CONDITIONNELS (AME)

Les effets marginaux moyens (AME) traduisent les coefficients probit en **différences de probabilité concrètes** entre hommes et femmes.

### Formule

$$\frac{\partial P(Y=1)}{\partial X_k} = \phi(\mathbf{X}'\boldsymbol{\beta}) \times (\beta_k + \gamma_k \times \text{Femme})$$

### 1. TEMPS DE TRAVAIL (TP) — Effet le plus différencié

| Variable | AME Hommes | AME Femmes | Δ (F-H) | IC 95% |
|----------|:----------:|:----------:|:-------:|:------:|
| **TP_2 (Temps partiel)** | **-0.57 pp** | **+2.23 pp** | **+2.80 pp** | [+2.31, +3.29] |

**Interprétation majeure** : C'est la **seule variable dont l'effet change de signe** entre hommes et femmes. 
- Pour les hommes, le temps partiel réduit légèrement la probabilité d'être transfrontalier (-0.57 pp)
- Pour les femmes, le temps partiel l'augmente significativement (+2.23 pp)

Le travail transfrontalier à temps partiel apparaît comme un **compromis spécifiquement féminin** entre revenus frontaliers et contraintes domestiques.

---

### 2. PARENTALITÉ (NENFR) — Pénalité croissante avec le nombre d'enfants

| Variable | AME Hommes | AME Femmes | Δ (F-H) | IC 95% |
|----------|:----------:|:----------:|:-------:|:------:|
| NENFR_1 (1 enfant) | -1.31 pp | -0.59 pp | **+0.72 pp** | [+0.28, +1.15] |
| NENFR_2 (2 enfants) | -1.26 pp | -1.20 pp | +0.06 pp | [-0.40, +0.53] |
| NENFR_3 (3 enfants) | -2.22 pp | -2.31 pp | -0.09 pp | [-0.79, +0.60] |
| NENFR_4 (4+ enfants) | -2.28 pp | -2.78 pp | **-0.49 pp** | [-1.66, +0.67] |
| NENFR_Z (enfants partis) | +0.56 pp | +1.69 pp | **+1.13 pp** | [+0.63, +1.63] |

**Interprétation** : Un pattern **non-linéaire** émerge :
- 1 enfant : les femmes sont **moins pénalisées** que les hommes (+0.72 pp)
- 2 enfants : effet **similaire** pour les deux genres
- 3-4+ enfants : la pénalité s'**intensifie pour les femmes** (-0.09 à -0.49 pp de plus)
- Enfants partis : les femmes bénéficient **davantage** de cette libération (+1.13 pp)

**Seuil critique** : La charge familiale ne devient différenciée selon le genre qu'au-delà de **2 enfants**.

---

### 3. CONDITIONS D'EMPLOI (EMPL) — Plafond de verre pour les indépendantes

| Variable | AME Hommes | AME Femmes | Δ (F-H) | IC 95% |
|----------|:----------:|:----------:|:-------:|:------:|
| EMPL_11 (Emplois aidés) | -9.11 pp | -10.02 pp | -0.91 pp | [-3.18, +1.36] |
| EMPL_12 (CDD) | +0.55 pp | +0.86 pp | +0.31 pp | [-0.36, +0.99] |
| EMPL_13 (Stagiaire) | -1.70 pp | -1.30 pp | +0.40 pp | [-2.22, +3.02] |
| EMPL_14 (Apprenti) | -0.41 pp | +0.72 pp | +1.13 pp | [-0.06, +2.32] |
| EMPL_15 (Intérim) | -0.86 pp | -0.83 pp | +0.03 pp | [-0.69, +0.75] |
| **EMPL_21 (Indépendant)** | **+9.01 pp** | **+3.98 pp** | **-5.03 pp** | [-6.27, -3.79] |
| EMPL_22 (Employeur) | -2.02 pp | -2.71 pp | -0.69 pp | [-2.03, +0.65] |
| EMPL_23 (Aide familial) | -1.10 pp | +0.16 pp | +1.26 pp | [-2.37, +4.89] |

**Résultat majeur** : L'écart de **-5.03 pp** pour les indépendants est le **plus important de toute l'analyse**. Les femmes entrepreneures bénéficient nettement moins des opportunités transfrontalières que leurs homologues masculins.

---

### 4. CAPITAL HUMAIN (DIPL) — Rendement différencié des diplômes

| Variable | AME Hommes | AME Femmes | Δ (F-H) | IC 95% |
|----------|:----------:|:----------:|:-------:|:------:|
| DIPL_3 (Aucun diplôme) | +1.35 pp | +0.93 pp | -0.42 pp | [-1.56, +0.73] |
| DIPL_11 (CEP) | +1.47 pp | +2.07 pp | +0.60 pp | [-0.96, +2.15] |
| DIPL_12 (Brevet) | -0.50 pp | +0.20 pp | +0.70 pp | [-0.21, +1.61] |
| DIPL_14 (Bac pro/techno) | -0.61 pp | -0.43 pp | +0.17 pp | [-0.40, +0.75] |
| DIPL_15 (Bac général) | -0.18 pp | -0.62 pp | -0.44 pp | [-1.00, +0.12] |
| DIPL_16 (Bac+2) | -0.83 pp | -0.33 pp | +0.51 pp | [-0.02, +1.03] |
| DIPL_17 (Bac+3/4) | +1.07 pp | +0.85 pp | -0.22 pp | [-0.79, +0.34] |
| **DIPL_18 (Bac+5)** | **+3.82 pp** | **+2.12 pp** | **-1.69 pp** | [-2.43, -0.96] |
| DIPL_19 (Grandes écoles) | +4.32 pp | +4.87 pp | +0.55 pp | [-0.81, +1.91] |

**Interprétation** : Un **plafond de verre** apparaît au niveau Bac+5 : les femmes très diplômées (+2.12 pp) bénéficient moins des opportunités transfrontalières que les hommes (+3.82 pp), soit un écart de **-1.69 pp**.

---

### 5. INTERSECTIONNALITÉ (DNAI) — Double avantage pour les femmes d'origine étrangère

| Variable | AME Hommes | AME Femmes | Δ (F-H) | IC 95% |
|----------|:----------:|:----------:|:-------:|:------:|
| DNAI_NEAUTREFR (Né autre région) | -2.40 pp | -2.75 pp | -0.35 pp | [-0.78, +0.08] |
| **DNAI_NEETRANGER (Né étranger)** | **+3.79 pp** | **+6.27 pp** | **+2.48 pp** | [+1.83, +3.13] |

**Interprétation** : Les femmes d'origine étrangère cumulent un **double avantage** dans l'accès au travail transfrontalier : elles bénéficient plus que les hommes de cette caractéristique (+2.48 pp de plus).

---

### 6. POSITION SOCIOPROFESSIONNELLE (GS)

| Variable | AME Hommes | AME Femmes | Δ (F-H) | IC 95% |
|----------|:----------:|:----------:|:-------:|:------:|
| GS_1 (Agriculteurs) | -0.91 pp | +0.07 pp | +0.97 pp | [-1.56, +3.51] |
| GS_2 (Artisans/Commerçants) | -2.87 pp | -3.30 pp | -0.43 pp | [-1.16, +0.31] |
| GS_3 (Cadres) | +1.06 pp | +0.44 pp | -0.62 pp | [-1.21, -0.04] |
| **GS_4 (Prof. intermédiaires)** | **+0.64 pp** | **+1.77 pp** | **+1.13 pp** | [+0.69, +1.56] |
| GS_6 (Ouvriers) | +1.88 pp | +2.65 pp | +0.77 pp | [+0.27, +1.27] |

**Interprétation** : Les femmes en professions intermédiaires et ouvrières bénéficient davantage du travail frontalier que les hommes. En revanche, les femmes cadres ont un avantage moindre (-0.62 pp).

---

### 7. SECTEUR D'ACTIVITÉ (NA5)

| Variable | AME Hommes | AME Femmes | Δ (F-H) | IC 95% |
|----------|:----------:|:----------:|:-------:|:------:|
| NA5_AZ (Agriculture) | -13.64 pp | -12.98 pp | +0.66 pp | [-2.84, +4.16] |
| NA5_BE (Industrie) | +0.67 pp | +0.65 pp | -0.02 pp | [-0.59, +0.54] |
| NA5_FZ (Construction) | -0.16 pp | +0.62 pp | +0.78 pp | [-0.12, +1.68] |
| **NA5_OQ (Administration)** | **-8.18 pp** | **-5.28 pp** | **+2.90 pp** | [+2.27, +3.53] |

**Interprétation** : L'administration publique pénalise moins les femmes que les hommes (+2.90 pp de différence). Cela peut refléter des conditions de travail plus compatibles avec les contraintes familiales dans le secteur public frontalier.

---

### 8. ACCÈS À LA MOBILITÉ (VOIT)

| Variable | AME Hommes | AME Femmes | Δ (F-H) | IC 95% |
|----------|:----------:|:----------:|:-------:|:------:|
| VOIT_0 (Aucune voiture) | -3.90 pp | -4.23 pp | -0.33 pp | [-1.23, +0.57] |
| **VOIT_2 (2 voitures)** | **+2.13 pp** | **+1.11 pp** | **-1.02 pp** | [-1.45, -0.60] |
| **VOIT_3 (3+ voitures)** | **+3.32 pp** | **+1.70 pp** | **-1.61 pp** | [-2.29, -0.94] |

**Interprétation** : La multi-motorisation du ménage bénéficie **davantage aux hommes**. Cela suggère une **répartition genrée de l'usage automobile** au sein du ménage : les hommes auraient priorité sur le(s) véhicule(s) pour le travail frontalier.

---

### 9. CYCLE DE VIE (AGEREV)

| Variable | AME Hommes | AME Femmes | Δ (F-H) | IC 95% |
|----------|:----------:|:----------:|:-------:|:------:|
| AGEREV_c (Âge centré) | +5.52 pp | +6.50 pp | +0.98 pp | [+0.54, +1.42] |
| AGEREV_c_sq (Âge² centré) | -5.08 pp | -6.42 pp | -1.34 pp | [-1.82, -0.86] |

**Interprétation** : Le profil âge-emploi est **plus prononcé pour les femmes** : elles bénéficient davantage de l'accumulation d'expérience mais subissent aussi une décroissance plus marquée après l'optimum.

---

### 10. ANCRAGE RÉSIDENTIEL (STOCD) — Groupe non validé

| Variable | AME Hommes | AME Femmes | Δ (F-H) | IC 95% |
|----------|:----------:|:----------:|:-------:|:------:|
| STOCD_21 (Locataire privé) | -0.79 pp | -0.52 pp | +0.28 pp | [-0.21, +0.76] |
| STOCD_22 (HLM) | -5.36 pp | -5.94 pp | -0.58 pp | [-1.41, +0.25] |
| STOCD_30 (Logé gratuit) | -0.53 pp | -1.05 pp | -0.52 pp | [-1.60, +0.56] |
| STOCD_Z (Non renseigné) | -0.55 pp | -0.28 pp | +0.27 pp | [-0.93, +1.48] |

**Interprétation** : L'effet du statut d'occupation du logement **ne diffère pas significativement** entre hommes et femmes. Les contraintes liées au patrimoine immobilier affectent de manière similaire les deux genres.

---

## SYNTHÈSE PAR DIMENSION THÉMATIQUE

| Dimension | Δ moyen | Direction | Interprétation |
|-----------|:-------:|:---------:|----------------|
| **Temps partiel** | **+2.80 pp** | ↑↑ F | Stratégie de conciliation féminine |
| **Secteur** | +1.37 pp | ↑ F | Administration favorable aux F |
| **Intersectionnalité** | +1.42 pp | ↑ F | Double avantage genre × origine |
| **Position sociale** | +1.64 pp | ↑ F | Sélection positive (prof. intermédiaires) |
| **Logement** | +1.16 pp | ↑ F | Ancrage moins contraignant |
| Parentalité | +0.26 pp | → | Pénalité croissante avec enfants |
| Diplôme | +0.24 pp | → | Plafond de verre Bac+5 |
| Emploi | -0.07 pp | → | Indépendance pénalise les F |
| **Mobilité** | **-0.65 pp** | ↓ F | Accès inégal au véhicule |

---

## RÉSULTATS ÉCONOMIQUES MAJEURS

### 1. Le temps partiel transfrontalier est une stratégie féminine

Le temps partiel est la **seule variable dont l'effet change de signe** entre hommes et femmes. Pour les hommes, il réduit la probabilité d'être transfrontalier (-0.57 pp) ; pour les femmes, il l'augmente (+2.23 pp). Ce résultat suggère que le travail frontalier à temps partiel constitue un **compromis spécifiquement féminin** permettant de concilier revenus transfrontaliers et contraintes domestiques.

### 2. Plafond de verre pour les diplômées et les indépendantes

| Variable | AME Hommes | AME Femmes | Δ (F-H) |
|----------|:----------:|:----------:|:-------:|
| Bac+5 | +3.82 pp | +2.12 pp | **-1.69 pp** |
| Indépendants | +9.01 pp | +3.98 pp | **-5.03 pp** |

Les femmes diplômées et entrepreneures bénéficient **moins des opportunités transfrontalières** que leurs homologues masculins. L'écart de **-5 pp** pour les indépendants est le plus important de toute l'analyse.

### 3. Parentalité : pénalité croissante avec le nombre d'enfants

| Nombre d'enfants | Δ (F-H) |
|------------------|:-------:|
| 1 enfant | +0.72 pp |
| 2 enfants | +0.06 pp |
| 3 enfants | -0.09 pp |
| 4+ enfants | -0.49 pp |

Un pattern **non-linéaire** émerge : la charge familiale ne devient différenciée selon le genre qu'au-delà de **2 enfants**. Au-delà de ce seuil, les femmes sont davantage pénalisées.

### 4. Intersectionnalité positive pour les femmes d'origine étrangère

| Variable | AME Hommes | AME Femmes | Δ (F-H) |
|----------|:----------:|:----------:|:-------:|
| Né étranger | +3.79 pp | +6.27 pp | **+2.48 pp** |

Les femmes d'origine étrangère cumulent un **double avantage** dans l'accès au travail transfrontalier.

### 5. Accès inégal au véhicule dans le ménage

| Motorisation | Δ (F-H) |
|--------------|:-------:|
| 2 voitures | -1.02 pp |
| 3+ voitures | -1.61 pp |

La multi-motorisation du ménage bénéficie **davantage aux hommes**, suggérant une répartition genrée de l'usage automobile.

---

## LIMITES ET PRÉCAUTIONS

### A. Interprétation causale

Les résultats documentent des **associations** et non des **effets causaux**. Plusieurs variables sont potentiellement endogènes :
- Le temps partiel peut être une **conséquence** du travail transfrontalier (choix d'organisation) plutôt qu'un déterminant
- La motorisation du ménage peut s'ajuster aux besoins de mobilité
- La catégorie socioprofessionnelle dépend en partie du marché du travail (français vs frontalier)

### B. Hétérogénéité non observée

Le modèle ne contrôle pas pour :
- Les préférences individuelles non observées
- La qualité des réseaux professionnels transfrontaliers
- Les compétences linguistiques (allemand, luxembourgeois, anglais)
- L'historique familial de travail frontalier

### C. Effets de composition

Les différences d'AME entre hommes et femmes reflètent à la fois :
- L'effet pur de l'interaction (coefficient γ)
- Les différences de distribution des covariables entre genres
- La non-linéarité du modèle probit

### D. Variable dépendante

Le modèle estime la probabilité d'**être** transfrontalier à un instant donné, pas celle de le **devenir**. Les résultats mélangent donc les déterminants de l'entrée et du maintien dans le travail transfrontalier.

---

## TRANSITION VERS LA PARTIE 5

### Pourquoi aller plus loin ?

La Partie 4 a montré que les déterminants du travail transfrontalier diffèrent selon le genre. Une question naturelle émerge : *ces différences reflètent-elles des arbitrages au sein des couples ?*

Plusieurs résultats suggèrent des mécanismes de coordination conjugale :
- L'effet différencié du temps partiel (conciliation)
- La pénalité parentale croissante pour les femmes (répartition des rôles)
- L'accès inégal au véhicule (partage des ressources du ménage)

### Objectif de la Partie 5

La Partie 5 testera explicitement la **corrélation des décisions** au sein des couples via un modèle **probit bivarié**. Ce modèle permet d'estimer simultanément :
- La probabilité que l'homme soit transfrontalier
- La probabilité que la femme soit transfrontalière
- Le coefficient de corrélation ρ entre ces deux décisions

### Questions de recherche

| Question | Paramètre clé |
|----------|---------------|
| Les décisions sont-elles coordonnées ? | ρ ≠ 0 |
| Coordination positive (les deux travaillent à l'étranger) ou négative (un seul) ? | Signe de ρ |
| Les déterminants diffèrent-ils entre équations ? | Coefficients β₁ vs β₂ |

---

## CONCLUSION

Cette analyse économétrique a démontré que le travail transfrontalier **n'est pas un phénomène neutre du point de vue du genre**. L'introduction de 45 termes d'interaction a permis de révéler une hétérogénéité substantielle des effets, validée statistiquement par des tests de Wald et du rapport de vraisemblance.

### Validation statistique

- **Test de Wald global** : χ² = 926.52, p < 10⁻¹⁶⁵ — interactions conjointement significatives
- **Test LR** : χ² = 934.07, p ≈ 0 — modèle avec interactions significativement meilleur
- **Tests par groupe** : 9/10 dimensions présentent des effets significativement différenciés
- **Critères d'information** : AIC et BIC améliorés malgré 45 paramètres supplémentaires

### Mécanismes identifiés

1. **Temps partiel comme stratégie féminine** : Seule variable dont l'effet change de signe entre genres
2. **Plafond de verre** : Les femmes diplômées et entrepreneures bénéficient moins des opportunités transfrontalières
3. **Pénalité parentale différenciée** : Effet de seuil au-delà de 2 enfants
4. **Intersectionnalité positive** : Double avantage pour les femmes d'origine étrangère
5. **Accès inégal au véhicule** : Répartition genrée de l'usage automobile dans le ménage

Ces résultats invitent à examiner comment ces arbitrages se jouent au niveau du couple — objet de la Partie 5.

---

**COMPLETED** :
- Création des 45 termes d'interaction SEXE × covariables
- Diagnostic et correction de la multicolinéarité (centrage AGEREV)
- Estimation du modèle probit avec interactions (MLE, convergence 10 itérations)
- Tests de significativité (Wald global, LR, Wald par groupe)
- Calcul des effets marginaux conditionnels (AME Hommes vs Femmes)
- Visualisations (6 graphiques)
- Interprétation économique par dimension thématique
- Documentation des limites méthodologiques

**PENDING** :
- Partie 5 : Probit bivarié (coordination des décisions au sein des couples)


# PARTIE 5 : MODÈLE PROBIT BIVARIÉ — DÉCISIONS CONJOINTES DES COUPLES

---

## OBJECTIF ET QUESTION DE RECHERCHE

Cette cinquième et dernière partie économétrique du projet teste l'**interdépendance des décisions professionnelles au sein des couples**. Contrairement aux modèles univariés (Parties 3 et 4), qui traitent chaque individu isolément, le probit bivarié modélise **simultanément** les choix des deux conjoints.

**Question centrale** : *Les choix professionnels des conjoints sont-ils coordonnés, substitutifs, ou indépendants ?*

**Hypothèses testées** :
- H₀ : ρ = 0 — Les décisions sont indépendantes (après contrôle des caractéristiques observables)
- H₁ : ρ ≠ 0 — Il existe une coordination (positive ou négative) des décisions

---

## DONNÉES ET RECONSTRUCTION DES COUPLES

### Pipeline de reconstruction

| Étape | Description | Effectif |
|-------|-------------|:--------:|
| Base initiale | Individus actifs occupés | 494 483 |
| Candidats | LPRM ∈ {1, 2} (REF ou CONJ) | 448 006 |
| Couples matchés (brut) | Appariement ID_FAMILLE | 126 974 |
| Couples filtrés (score ≥ 4) | Contrôle de qualité | **126 935** |
| Taux de conservation | | 100.0% |

### Critères de sélection des couples

| Critère | Justification |
|---------|---------------|
| Exactement 1 individu avec LPRM = 1 | Une seule personne de référence |
| Exactement 1 individu avec LPRM = 2 | Un(e) seul(e) conjoint(e) |
| NUMMI ≠ 'Z' | Ménage identifiable |
| Les deux membres sont actifs occupés | Décision de travail transfrontalier possible |
| Couple hétérosexuel | Interprétation H/F non ambiguë |

### Identifiant de famille

$$\text{ID\_FAMILLE} = \text{ID\_ZONE} + \text{NUMMI} + \text{NUMF}$$

### Format des données : WIDE

| Type de variable | Exemples | Suffixe |
|------------------|----------|---------|
| Caractéristiques de REF | AGEREV_REF, DIPL_REF, GS_REF | `_REF` |
| Caractéristiques de CONJ | AGEREV_CONJ, DIPL_CONJ, GS_CONJ | `_CONJ` |
| Variables du ménage | DEPT, VOIT, STOCD, NENFR | (pas de suffixe) |
| Variables dépendantes | Y_REF, Y_CONJ | — |

### Score de qualité

- **Score moyen** : 4.98 / 5
- **Cohérence déclarative** (COUPLE = 1) : 99.55%

---

## STATISTIQUES DESCRIPTIVES PRÉLIMINAIRES

### Distribution des décisions au sein des couples

|  | Femme non-transf. (Y_CONJ=0) | Femme transf. (Y_CONJ=1) | Total |
|--|:----------------------------:|:------------------------:|:-----:|
| **Homme non-transf. (Y_REF=0)** | 108 827 (85.7%) | 5 030 (4.0%) | 113 857 (89.7%) |
| **Homme transf. (Y_REF=1)** | 7 003 (5.5%) | 6 075 (4.8%) | 13 078 (10.3%) |
| **Total** | 115 830 (91.2%) | 11 105 (8.8%) | 126 935 (100%) |

### Test informel de corrélation

| Indicateur | Valeur |
|------------|:------:|
| P(Y_REF = 1) | 10.30% |
| P(Y_CONJ = 1) | 8.75% |
| P(1,1) sous indépendance | 0.90% |
| P(1,1) observée | **4.79%** |
| **Ratio observé/indépendance** | **5.3×** |

→ **Forte présomption de corrélation POSITIVE (ρ > 0)**

### Coefficient Phi brut

**Phi = 0.452** — Corrélation remarquablement élevée pour des variables binaires (> 0.3 = forte association).

Cette corrélation brute combine :
1. L'homogamie observée (caractéristiques communes)
2. La localisation commune (même département)
3. La coordination inobservée (à tester via ρ)

---

## SPÉCIFICATION DU MODÈLE

### Formulation mathématique

Le probit bivarié modélise deux variables latentes simultanées :

$$
\begin{cases}
Y_{REF}^* = X_{REF}'\beta_{REF} + Z'\gamma_{REF} + \varepsilon_{REF} & \text{(équation de la personne de référence)} \\
Y_{CONJ}^* = X_{CONJ}'\beta_{CONJ} + Z'\gamma_{CONJ} + \varepsilon_{CONJ} & \text{(équation du conjoint)}
\end{cases}
$$

avec :
- $X_{REF}$, $X_{CONJ}$ : caractéristiques **individuelles** de chaque conjoint
- $Z$ : caractéristiques **du ménage** (communes aux deux équations)
- $\beta_{REF}$, $\beta_{CONJ}$ : coefficients spécifiques à chaque conjoint
- $\gamma_{REF}$, $\gamma_{CONJ}$ : coefficients des variables ménage

### Distribution jointe des erreurs

$$
\begin{pmatrix} \varepsilon_{REF} \\ \varepsilon_{CONJ} \end{pmatrix} \sim \mathcal{N}\left(\begin{pmatrix} 0 \\ 0 \end{pmatrix}, \begin{pmatrix} 1 & \rho \\ \rho & 1 \end{pmatrix}\right)
$$

### Variables observées

$$
Y_{REF} = \mathbb{1}_{Y_{REF}^* > 0} \quad \text{et} \quad Y_{CONJ} = \mathbb{1}_{Y_{CONJ}^* > 0}
$$

### Vraisemblance

Pour chaque couple $i$, la contribution à la log-vraisemblance dépend du quadrant $(Y_{REF}, Y_{CONJ})$ :

$$
\mathcal{L}_i = \begin{cases}
\Phi_2(X_{REF}'\beta_{REF}, X_{CONJ}'\beta_{CONJ}; \rho) & \text{si } (Y_{REF}, Y_{CONJ}) = (1, 1) \\
\Phi_2(X_{REF}'\beta_{REF}, -X_{CONJ}'\beta_{CONJ}; -\rho) & \text{si } (Y_{REF}, Y_{CONJ}) = (1, 0) \\
\Phi_2(-X_{REF}'\beta_{REF}, X_{CONJ}'\beta_{CONJ}; -\rho) & \text{si } (Y_{REF}, Y_{CONJ}) = (0, 1) \\
\Phi_2(-X_{REF}'\beta_{REF}, -X_{CONJ}'\beta_{CONJ}; \rho) & \text{si } (Y_{REF}, Y_{CONJ}) = (0, 0)
\end{cases}
$$

où $\Phi_2(\cdot, \cdot; \rho)$ désigne la fonction de répartition de la loi normale bivariée avec corrélation $\rho$.

### Interprétation du paramètre ρ

| Valeur de ρ | Interprétation | Mécanisme économique |
|:-----------:|----------------|----------------------|
| **ρ > 0** | Corrélation positive | Coordination : les deux conjoints tendent à être transfrontaliers ensemble |
| **ρ < 0** | Corrélation négative | Substitution : si l'un est transfrontalier, l'autre tend à ne pas l'être |
| **ρ = 0** | Indépendance | Pas de coordination — décisions individuelles |

---

## VARIABLES EXPLICATIVES

### Pourquoi ne pas utiliser les interactions de la Partie 4 ?

Le probit bivarié capture **nativement** l'hétérogénéité de genre :

| Approche | Structure | Capture de l'hétérogénéité |
|----------|-----------|---------------------------|
| **Partie 4** (Probit poolé + interactions) | 1 équation avec termes SEXE × X | Via les coefficients d'interaction γ |
| **Partie 5** (Probit bivarié) | 2 équations séparées | Via β_REF ≠ β_CONJ directement |

Les interactions SEXE × covariables seraient **redondantes** dans le bivarié. On utilise donc les **17 variables de base** validées en Partie 3.

### Variables individuelles (dupliquées pour REF et CONJ)

| Variable | Description | Modalités | Référence |
|----------|-------------|:---------:|-----------|
| AGEREV | Âge révolu | Continue | — |
| AGEREV² | Terme quadratique | Continue | — |
| SEXE | Genre | 2 | Homme (1) |
| DIPL | Diplôme le plus élevé | 12 | CAP/BEP |
| GS | Groupe socioprofessionnel | 6 | Employés |
| EMPL | Condition d'emploi | 9 | CDI/Titulaire |
| NA5 | Secteur d'activité | 5 | Services |
| DNAI | Lieu de naissance | 3 | Né Grand Est |
| INATC | Nationalité | 2 | Français |
| TP | Temps de travail | 2 | Temps complet |
| ETUD | Inscription aux études | 2 | Non inscrit |

### Variables du ménage (communes aux deux équations)

| Variable | Description | Modalités | Référence |
|----------|-------------|:---------:|-----------|
| DEPT | Département de résidence | 10 | Marne (51) |
| NENFR | Nombre d'enfants | 6 | 0 enfant |
| VOIT | Nombre de voitures | 4 | 1 voiture |
| STOCD | Statut d'occupation | 5 | Propriétaire |
| TYPL | Type de logement | 6 | Maison |
| SANI | Installations sanitaires | 4 | Standard |

### Nombre de paramètres

| Composante | Total |
|------------|:-----:|
| Équation REF | ~57 coefficients |
| Équation CONJ | ~57 coefficients |
| Corrélation ρ | 1 |
| **Total** | **~115 paramètres** |

---

## RÉSULTATS D'ESTIMATION

### Statistiques globales

| Indicateur | Valeur |
|------------|:------:|
| Observations (couples) | 126 935 |
| Paramètres estimés | 115 |
| Mode d'estimation | FULL |
| Temps d'estimation | 3 312.7 secondes (~55 min) |
| Convergence | ✅ Oui |

### Qualité d'ajustement des probits univariés

| Équation | Log-vraisemblance | Pseudo-R² |
|----------|:-----------------:|:---------:|
| Probit REF | -31 819.00 | 0.2433 |
| Probit CONJ | -28 105.81 | 0.2523 |

### Comparaison des log-vraisemblances

| Modèle | Log-vraisemblance |
|--------|:-----------------:|
| Probits indépendants (ρ = 0) | -60 014.96 |
| Probit bivarié (ρ libre) | -56 244.42 |
| **Amélioration** | **+3 770.53** |

L'amélioration de la log-vraisemblance de près de 3 770 points indique un gain d'ajustement considérable du modèle bivarié par rapport à l'hypothèse d'indépendance.

### PARAMÈTRE CENTRAL : ρ = 0.64

| Paramètre | Estimation | Écart-type | IC 95% |
|-----------|:----------:|:----------:|:------:|
| **ρ** | **0.6423** | 0.0117 | [0.619, 0.665] |

### Tests d'hypothèse H₀ : ρ = 0

| Test | Statistique | P-value | Conclusion |
|------|:-----------:|:-------:|------------|
| **Ratio de vraisemblance (LR)** | χ²₁ = **7 541.06** | < 10⁻¹⁰ | H₀ rejetée *** |
| **Wald** | z = 54.67 | < 10⁻¹⁰ | H₀ rejetée *** |

Les deux tests rejettent **massivement** l'hypothèse d'indépendance. La corrélation estimée est hautement significative au seuil de 0.1%.

---

## INTERPRÉTATION STATISTIQUE

### Signification de ρ = 0.64

Le coefficient de corrélation ρ = 0.64 mesure l'association entre les résidus des deux équations :

> **Après avoir contrôlé** le département de résidence, le niveau de diplôme, la catégorie socioprofessionnelle, le secteur d'activité, l'âge, le type de logement, le nombre d'enfants, et toutes les autres variables du modèle, **il subsiste une corrélation forte et positive** entre la propension au travail transfrontalier des deux conjoints.

### Comparaison avec la corrélation brute

**Fait contre-intuitif** : ρ (0.64) > Phi brut (0.45)

Cela signifie que les variables de contrôle (département, diplôme, etc.) **masquaient partiellement** la vraie corrélation entre les décisions. L'homogamie observée ne capture qu'une partie de la coordination réelle des choix professionnels.

### Robustesse du résultat

- **Intervalle de confiance étroit** : [0.619, 0.665] — estimation très précise
- **Double validation** : Tests LR et Wald concordants
- **Grande taille d'échantillon** : 126 935 couples garantit la puissance statistique
- **Convergence atteinte** : L'algorithme a convergé vers un optimum stable

---

## COMPARAISON DES COEFFICIENTS REF vs CONJ (Section 5)

### Résultats globaux

| Indicateur | Valeur |
|------------|:------:|
| Paires de variables comparées | 30 |
| Différences significatives (p < 0.05) | 9 (30%) |
| Variables symétriques (p ≥ 0.10) | 20 (67%) |
| **Corrélation globale β_REF / β_CONJ** | **0.921** |

La corrélation de 0.92 indique une **très forte similarité** des structures de coefficients.

### Variables asymétriques significatives (p < 0.05)

#### Secteur d'activité (2 variables)

| Variable | β_REF | β_CONJ | Diff | P-value |
|----------|:-----:|:------:|:----:|:-------:|
| **NA5_OQ** (public) | -0.671 | -0.559 | -0.111 | <0.001 *** |
| **NA5_BE** (industrie) | -0.029 | +0.035 | -0.064 | 0.005 ** |

Le secteur public pénalise **davantage REF que CONJ**.

#### Origine géographique (2 variables)

| Variable | β_REF | β_CONJ | Diff | P-value |
|----------|:-----:|:------:|:----:|:-------:|
| **DNAI_3** (né étranger) | +0.237 | +0.351 | -0.114 | <0.001 *** |
| **DNAI_2** (né hors GE) | -0.115 | -0.051 | -0.064 | 0.017 * |

Être né à l'étranger booste **plus fortement CONJ que REF**.

#### Temps de travail (1 variable)

| Variable | β_REF | β_CONJ | Diff | P-value |
|----------|:-----:|:------:|:----:|:-------:|
| **TP_2** (temps partiel) | +0.094 | +0.181 | -0.088 | <0.001 *** |

Le temps partiel a un effet positif **plus fort pour CONJ** → stratégie de spécialisation intra-ménage.

#### CSP (2 variables)

| Variable | β_REF | β_CONJ | Diff | P-value |
|----------|:-----:|:------:|:----:|:-------:|
| **GS_3** (prof. interm.) | -0.129 | -0.037 | -0.092 | 0.003 ** |
| **GS_4** (employés) | -0.095 | -0.023 | -0.072 | 0.003 ** |

#### Genre et contrat

| Variable | β_REF | β_CONJ | Diff | P-value |
|----------|:-----:|:------:|:----:|:-------:|
| **SEXE_2** (femme) | -0.147 | -0.191 | +0.044 | 0.038 * |
| **EMPL_15** (intérim) | -0.044 | -0.179 | +0.135 | 0.001 ** |

Être femme pénalise **moins REF que CONJ**. Une femme désignée comme personne de référence est en position professionnelle plus affirmée.

### Synthèse par catégorie

| Catégorie | N vars | Signif. | Pattern |
|-----------|:------:|:-------:|---------|
| SEXE | 1 | 1 | REF > CONJ |
| ÂGE | 2 | 0 | Symétrique |
| DIPLÔME | 5 | 0 | Symétrique |
| CSP | 5 | 2 | REF < CONJ |
| CONTRAT | 8 | 1 | Variable |
| SECTEUR | 4 | 2 | REF < CONJ |
| ORIGINE | 3 | 2 | REF < CONJ |
| TEMPS | 2 | 1 | REF < CONJ |

### Variables parfaitement symétriques

| Variable | β_REF | β_CONJ | P-value | Interprétation |
|----------|:-----:|:------:|:-------:|----------------|
| INATC_2 (étranger) | +0.522 | +0.523 | 0.982 | Parfaitement symétrique |
| DIPL_BAC5 | +0.196 | +0.192 | 0.934 | Même rendement du diplôme |
| GS_6 (ouvriers) | +0.120 | +0.123 | 0.914 | Même effet CSP |

**Conclusion** : Le diplôme, la nationalité et le type de contrat influencent la probabilité de travail transfrontalier de manière **identique** pour REF et CONJ.

---

## EFFETS MARGINAUX MOYENS (Section 6)

### Calcul des AME bivariés

Dans le cadre bivarié, les AME intègrent la corrélation ρ entre les deux équations :

$$\text{AME}_{1,k} = \mathbb{E}\left[\phi(\eta_1) \cdot \Phi\left(\frac{\eta_2 - \rho \eta_1}{\sqrt{1-\rho^2}}\right)\right] \cdot \beta_{1,k}$$

| Équation | Facteur AME |
|----------|:-----------:|
| REF | 0.0424 |
| CONJ | 0.0372 |

### TOP 15 des effets marginaux — Équation REF

| Variable | Coefficient | AME (pp) |
|----------|:-----------:|:--------:|
| DEPT_57 (Moselle) | +1.4638 | +6.20% |
| DEPT_68 (Haut-Rhin) | +1.2098 | +5.13% |
| DEPT_54 (Meurthe-Mos.) | +0.9093 | +3.85% |
| NA5_OQ (public) | -0.6705 | -2.84% |
| EMPL_21 (CDD) | -0.5954 | -2.52% |
| INATC_2 (étranger) | +0.5218 | +2.21% |
| DEPT_67 (Bas-Rhin) | +0.5094 | +2.16% |
| EMPL_22 (interim) | -0.4798 | -2.03% |
| EMPL_11 (indépendant) | -0.4168 | -1.77% |
| GS_2 (artisans) | -0.3138 | -1.33% |
| GS_1 (agriculteurs) | -0.2597 | -1.10% |
| DNAI_3 (né étranger) | +0.2371 | +1.01% |
| EMPL_14 (employeur) | -0.2236 | -0.95% |
| DIPL_BAC5 | +0.1963 | +0.83% |
| AGEREV | +0.0172 | +0.07% |

### TOP 15 des effets marginaux — Équation CONJ

| Variable | Coefficient | AME (pp) |
|----------|:-----------:|:--------:|
| DEPT_57 (Moselle) | +1.6154 | +6.01% |
| DEPT_68 (Haut-Rhin) | +1.3066 | +4.86% |
| NA5_AZ (public admin) | -1.0988 | -4.09% |
| DEPT_54 (Meurthe-Mos.) | +1.0013 | +3.73% |
| EMPL_21 (CDD) | -0.6180 | -2.30% |
| NA5_OQ (public) | -0.5592 | -2.08% |
| INATC_2 (étranger) | +0.5227 | +1.94% |
| DEPT_67 (Bas-Rhin) | +0.4951 | +1.84% |
| EMPL_22 (interim) | -0.4082 | -1.52% |
| EMPL_23 | -0.3588 | -1.33% |
| DNAI_3 (né étranger) | +0.3507 | +1.30% |
| GS_2 (artisans) | -0.2596 | -0.97% |
| DIPL_SANS_DIPL | -0.2309 | -0.86% |
| EMPL_11 (indépendant) | -0.2413 | -0.90% |
| DIPL_BAC5 | +0.1919 | +0.71% |

### Déterminants cohérents entre les deux équations

Les principaux déterminants sont identiques :
- **Géographie** : Moselle, Haut-Rhin, Meurthe-et-Moselle (+3 à +6 pp)
- **Secteur public** : Effet négatif fort (-2 à -4 pp)
- **Nationalité étrangère** : Effet positif (+2 pp)
- **Type de contrat** : CDD et intérim pénalisent (-1.5 à -2.5 pp)
- **Diplôme élevé** : Bac+5 favorise (+0.7 à +0.8 pp)

---

## PROBABILITÉS CONDITIONNELLES (Section 7)

### Probabilités jointes et marginales

**Probabilités marginales** :
- P(REF transfrontalier) = 10.30%
- P(CONJ transfrontalier) = 8.75%

**Probabilités jointes** :

|  | CONJ = 1 | CONJ = 0 | Total |
|--|:--------:|:--------:|:-----:|
| **REF = 1** | 4.91% | 5.39% | 10.30% |
| **REF = 0** | 3.84% | 85.86% | 89.70% |
| **Total** | 8.75% | 91.25% | 100% |

### PROBABILITÉS CONDITIONNELLES COMPLÈTES

#### Effet du CONJOINT sur REF

| Condition | Probabilité | vs Marginal |
|-----------|:-----------:|:-----------:|
| P(REF=1 \| CONJ=1) | **47.50%** | +37.2 pp |
| P(REF=1 \| CONJ=0) | **6.59%** | -3.7 pp |
| **Effet multiplicateur** | **×4.72** | |

#### Effet de REF sur CONJOINT

| Condition | Probabilité | vs Marginal |
|-----------|:-----------:|:-----------:|
| P(CONJ=1 \| REF=1) | **47.66%** | +38.9 pp |
| P(CONJ=1 \| REF=0) | **6.55%** | -2.2 pp |
| **Effet multiplicateur** | **×4.78** | |

### EFFET MULTIPLICATEUR ×4.7

> **Un individu dont le conjoint est transfrontalier a près d'une chance sur deux (47.5%) d'être lui-même transfrontalier**, contre seulement 6.6% si le conjoint ne l'est pas.

### Écarts conditionnels

| Mesure | Valeur |
|--------|:------:|
| P(REF=1\|CONJ=1) - P(REF=1\|CONJ=0) | **+40.91 pp** |
| P(CONJ=1\|REF=1) - P(CONJ=1\|REF=0) | **+41.11 pp** |

**Interprétation** :
- Avoir un conjoint transfrontalier augmente la probabilité d'être soi-même transfrontalier de **41 points de pourcentage**
- L'effet est **parfaitement symétrique** entre les deux membres du couple

### Odds ratio

$$\text{Odds ratio} = \frac{P(1,1) \times P(0,0)}{P(1,0) \times P(0,1)} = \mathbf{12.8}$$

**Les couples bi-transfrontaliers sont 12.8 fois plus fréquents que si les décisions étaient indépendantes.**

---

## MÉCANISMES EXPLICATIFS

### 1. Effets de réseau et transmission d'information

Un conjoint déjà transfrontalier réduit les **coûts d'entrée** pour l'autre :
- Information sur les opportunités d'emploi
- Connaissance des procédures administratives (fiscalité, sécurité sociale)
- Réseau professionnel mobilisable
- Maîtrise des spécificités culturelles et linguistiques

### 2. Préférences communes non observées

Les couples partagent des préférences non capturées par les variables du modèle :
- **Tolérance aux trajets longs** : accepter 1h+ de trajet quotidien
- **Appétence pour le différentiel salarial** : arbitrage revenu vs qualité de vie
- **Ouverture internationale** : bilinguisme, mobilité antérieure
- **Aversion au risque similaire** : emploi à l'étranger perçu comme plus incertain

L'appariement conjugal (assortative mating) sur ces dimensions génère une corrélation résiduelle.

### 3. Coordination stratégique explicite

La coordination peut être **stratégique et explicite** :
- **Covoiturage** : économie de temps et d'argent
- **Synchronisation des horaires** : départs et retours communs
- **Garde d'enfants** : organisation facilitée si les deux parents ont des horaires similaires
- **Mutualisation des réseaux** : garde partagée avec d'autres couples transfrontaliers

### 4. Localisation résidentielle endogène

Au-delà du département (contrôlé), la **localisation fine** détermine conjointement les opportunités :
- Un couple qui choisit de s'installer à 5 km de la frontière luxembourgeoise fait face aux mêmes opportunités transfrontalières pour les deux membres
- Ce choix résidentiel est lui-même potentiellement motivé par le projet professionnel du couple

### 5. Chocs communs au ménage

Des événements affectant le ménage dans son ensemble peuvent déclencher simultanément les deux décisions :
- Achat immobilier près de la frontière
- Restructuration d'un employeur local
- Naissance d'un enfant modifiant l'arbitrage temps/revenu

---

## IMPLICATIONS POUR LES POLITIQUES PUBLIQUES

### 1. Effet multiplicateur des politiques d'accompagnement

L'effet ×4.7 implique que toute politique facilitant le travail transfrontalier aura des **effets démultiplicateurs** :
- Attirer un travailleur transfrontalier → forte probabilité que son conjoint suive
- Les dispositifs d'accompagnement devraient cibler les **ménages** plutôt que les individus

### 2. Infrastructures de transport

La concentration des couples bi-transfrontaliers justifie des investissements ciblés :
- **Parkings relais** : dimensionnés pour des flux importants
- **Transports en commun** : lignes transfrontalières à haute fréquence
- **Covoiturage organisé** : plateformes dédiées aux frontaliers

### 3. Services aux familles

Les couples bi-transfrontaliers cumulent les contraintes horaires :
- **Crèches et écoles** : horaires élargis (6h-20h)
- **Services périscolaires** : adaptés aux trajets longs
- **Télétravail** : particulièrement bénéfique pour ces ménages

### 4. Fiscalité et protection sociale

Les couples bi-transfrontaliers sont exposés à une complexité administrative accrue :
- Deux pays d'emploi potentiellement différents
- Questions de résidence fiscale et de convention bilatérale
- Droits sociaux (retraite, chômage) dans plusieurs systèmes

Une simplification administrative bénéficierait particulièrement à ces ménages.

---

## LIMITES MÉTHODOLOGIQUES

### A. Corrélation vs Causalité

L'effet multiplicateur de ×4.7 mesure une **association**, pas un effet causal. On ne peut pas distinguer :
- **Effet causal direct** : le statut du conjoint cause la décision de l'autre
- **Sélection sur inobservables** : les couples se forment sur des préférences communes

Une identification causale nécessiterait des variations exogènes (fermeture d'entreprise, ouverture de ligne de transport).

### B. Hétérogénéité non modélisée

Le modèle suppose un ρ **unique** pour tous les couples. L'effet pourrait varier selon :
- La présence d'enfants (contraintes vs incitations financières)
- Le département (proximité de la frontière)
- Le pays de destination (Luxembourg vs Suisse vs Allemagne)
- L'ancienneté dans le statut transfrontalier

Des extensions avec ρ variable permettraient d'explorer cette hétérogénéité.

### C. Biais de sélection des couples

L'échantillon ne contient que les couples où **les deux membres sont actifs occupés**. Les couples avec un conjoint inactif ou au chômage sont exclus, ce qui peut affecter la généralisabilité des résultats.

### D. Variable dépendante

Comme pour les parties précédentes, le modèle estime la probabilité d'**être** transfrontalier (stock), pas de le **devenir** (flux). Les résultats mélangent déterminants de l'entrée et du maintien.

---

## ANNEXE TECHNIQUE : PROCÉDURE D'ESTIMATION

### Algorithme d'optimisation

| Paramètre | Valeur |
|-----------|--------|
| Algorithme | L-BFGS-B |
| Valeur initiale ρ | 0.6 (basée sur Phi observé) |
| Critère de convergence | gtol = 1e-5 |
| Itérations max | 200 |
| Convergence atteinte | Oui |

### CDF normale bivariée

Implémentation de la méthode de Drezner-Wesolowsky (1990) avec quadrature de Gauss-Legendre :
- 12 points pour |ρ| < 0.75
- 20 points pour |ρ| ≥ 0.75

### Écarts-types des coefficients

Approximation par probits univariés estimés par Newton-Raphson (statsmodels). Cette approche en deux étapes est conservative : elle tend à surestimer les SE.

---

## TABLEAU RÉCAPITULATIF — PARTIE V

| Section | Question | Résultat principal |
|---------|----------|-------------------|
| 1-2 | Construction de la base couples | 126 935 couples actifs |
| 3 | Préparation des variables | Matrices X_REF, X_CONJ, Z |
| 4 | Estimation du probit bivarié | **ρ = 0.64** (p < 10⁻¹⁰) |
| 5 | Comparaison coefficients | Corrélation β = 0.92 |
| 6 | Effets marginaux | AME bivariés calculés |
| 7 | Probabilités conditionnelles | **Effet ×4.7** |

---

## CONCLUSION

> L'estimation du probit bivarié établit de manière robuste l'existence d'une **coordination intra-ménage** dans les décisions de travail transfrontalier. La corrélation résiduelle de **ρ = 0.64**, traduite par un effet multiplicateur de **×4.7**, démontre que les choix professionnels des deux membres du couple ne peuvent être analysés indépendamment.

### Résultats majeurs

| Résultat | Valeur | Message économique |
|----------|:------:|-------------------|
| Corrélation résiduelle ρ | **0.64** | Coordination substantielle après contrôle |
| Test LR (ρ = 0) | **7 541** | Rejet massif de l'indépendance |
| Effet multiplicateur | **×4.7** | Avoir un conjoint transf. multiplie par 5 sa propre probabilité |
| Écart conditionnel | **+41 pp** | L'effet absolu est massif (de 6.6% à 47.5%) |
| Symétrie | Parfaite | La coordination est bidirectionnelle |
| Odds ratio | **12.8** | Les couples bi-transf. sont très surreprésentés |
| Corrélation β_REF/β_CONJ | **0.92** | Déterminants très similaires |

### Message central

Les couples bi-transfrontaliers, **12.8 fois plus fréquents qu'attendu sous indépendance**, constituent une catégorie spécifique de ménages dont les contraintes et les stratégies méritent une attention particulière des politiques publiques. La modélisation des choix professionnels transfrontaliers doit intégrer la **dimension familiale** : les décisions ne sont pas prises individuellement, mais s'inscrivent dans une **logique de ménage**.

---

## PROLONGEMENTS POSSIBLES

- **Modèle dynamique** : Étudier l'ordre des transitions (qui devient transfrontalier en premier ?)
- **Hétérogénéité de ρ** : Estimer ρ par sous-groupe (département, présence d'enfants)
- **Identification causale** : Exploiter des chocs exogènes (fermeture d'entreprise, ouverture de ligne de transport)
- **Biprobit avec variables instrumentales** : Corriger l'endogénéité potentielle de certaines covariables

---

*Rapport rédigé le 9 janvier 2026 — Projet INSEE Mobilité Transfrontalière Grand Est*  
*Romain Mehdi FEHRI*  
*Master 2 Statistique, Économétrie & Data Science — Université de Strasbourg*